# 1. Import Packages for the Environment

In [1501]:
# Import basic packages for later use
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch

In [1502]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [1503]:
IN_KAGGLE = os.path.exists('/kaggle/input')
IN_COLAB = not IN_KAGGLE and os.path.exists('/content')
IN_DEIB = not IN_KAGGLE and not IN_COLAB

In [1504]:
if not IN_DEIB:
    !pip install nnunetv2
    !pip install captum

# 2. Mount the dataset

In [1505]:
from batchgenerators.utilities.file_and_folder_operations import join

if IN_COLAB:
    # Google Colab
    # for colab users only - mounting the drive

    from google.colab import drive
    drive.mount('/content/drive',force_remount = True)

    drive_dir = "/content/drive/My Drive"
    mount_dir = join(drive_dir, "tesi", "automi")
    base_dir = os.getcwd()
elif IN_KAGGLE:
    # Kaggle
    mount_dir = "/kaggle/input/automi-seg"
    base_dir = os.getcwd()
    print(base_dir)
    !ls '/kaggle/input'
    !cd "/kaggle/input/automi-seg" ; ls
else:
    mount_dir = "/workspace/data"
    base_dir = "/workspace/output"
    os.chdir(base_dir)
    print("base_dir:", base_dir)

base_dir: /workspace/output


# 3. Setting up nnU-Nets folder structure and environment variables
nnUnet expects a certain folder structure and environment variables.

Roughly they tell nnUnet:
1. Where to look for stuff
2. Where to put stuff

For more information about this please check: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/setting_up_paths.md

## 3.1 Set environment Variables and creating folders

In [1506]:
# ===========================
# 📦 SETUP nnUNet ENVIRONMENT
# ===========================

# Definisci i path da settare
path_dict = {
    "nnUNet_raw": join(mount_dir, "nnunet_raw"),
    "nnUNet_preprocessed": join(mount_dir, "preprocessed_files"),#"nnUNet_preprocessed"),
    "nnUNet_results": join(mount_dir, "results"),#"nnUNet_results"),
    # "RAW_DATA_PATH": join(mount_dir, "RawData"),  # Facoltativo, se ti serve salvare zips
}

# Scrivi i path nelle variabili di ambiente, che vengono lette dal modulo paths di nnunetv2
for env_var, path in path_dict.items():
    os.environ[env_var] = path

from nnunetv2.paths import nnUNet_results, nnUNet_raw

if IN_KAGGLE:
    if nnUNet_raw == None:
        nnUNet_raw = "/kaggle/input/nnunet_raw"
    if nnUNet_results == None:
        nnUNet_results = "/kaggle/input/results"
    # Kaggle has some very unconsistent behaviors in dataset mounting...
    #nnUNet_raw = "/kaggle/input/automi-seg/nnunet_raw"
    #nnUNet_results = "/kaggle/input/automi-seg/results"
    
print("nnUNet_raw:", nnUNet_raw)
print("nnUNet_results:", nnUNet_results)

nnUNet_raw: /workspace/data/nnunet_raw
nnUNet_results: /workspace/data/results


### Some tests

In [1507]:
# all volumes of fold 0 test set
volume_codes = ["00004", "00005", "00024", "00027", "00029", "00034", "00039", "00044"]

In [1508]:
ct_img_paths = {}
organ_map_paths = {}

for volume_code in volume_codes:
    if IN_KAGGLE:
        ct_img_paths[volume_code] = join(nnUNet_raw, "imagesTr", f"AUTOMI_{volume_code}_0000.nii")
        organ_map_paths[volume_code] = join(nnUNet_raw, "total_segmentator_structures", f"AUTOMI_{volume_code}_0000", "mask_mask_add_input_20_total_segmentator.nii")
    else:
        ct_img_paths[volume_code] = join(nnUNet_raw, "imagesTr", f"AUTOMI_{volume_code}_0000.nii.gz")
        organ_map_paths[volume_code] = join(nnUNet_raw, "total_segmentator_structures", f"AUTOMI_{volume_code}_0000", "mask_mask_add_input_20_total_segmentator.nii.gz")
    ct_img = nib.load(ct_img_paths[volume_code])
    organ_map = nib.load(organ_map_paths[volume_code])
    print(f"Volume {volume_code}:")
    print("CT shape:", ct_img.shape)
    print("Organ shape:", organ_map.shape)
    print("Spacing:", ct_img.header.get_zooms())
    print("Organ spacing:", organ_map.header.get_zooms())
    assert np.all(ct_img.affine == organ_map.affine), "CT and organ mask affine matrices do not match!"

Volume 00004:
CT shape: (512, 512, 221)
Organ shape: (512, 512, 221)
Spacing: (1.3671875, 1.3671875, 5.0)
Organ spacing: (1.3671875, 1.3671875, 5.0)
Volume 00005:
CT shape: (512, 512, 219)
Organ shape: (512, 512, 219)
Spacing: (1.171875, 1.171875, 5.0)
Organ spacing: (1.171875, 1.171875, 5.0)
Volume 00024:
CT shape: (512, 512, 321)
Organ shape: (512, 512, 321)
Spacing: (0.976562, 0.976562, 3.0)
Organ spacing: (0.976562, 0.976562, 3.0)
Volume 00027:
CT shape: (512, 512, 236)
Organ shape: (512, 512, 236)
Spacing: (1.3671875, 1.3671875, 5.0)
Organ spacing: (1.3671875, 1.3671875, 5.0)
Volume 00029:
CT shape: (512, 512, 259)
Organ shape: (512, 512, 259)
Spacing: (1.367188, 1.367188, 7.5)
Organ spacing: (1.367188, 1.367188, 7.5)
Volume 00034:
CT shape: (512, 512, 249)
Organ shape: (512, 512, 249)
Spacing: (1.171875, 1.171875, 5.0)
Organ spacing: (1.171875, 1.171875, 5.0)
Volume 00039:
CT shape: (512, 512, 283)
Organ shape: (512, 512, 283)
Spacing: (1.171875, 1.171875, 5.0)
Organ spacing: (1.

In [1509]:
ct_img = nib.load(ct_img_paths[volume_codes[1]])
print("affine:", ct_img.affine)

affine: [[-1.17187500e+00  0.00000000e+00  0.00000000e+00  3.00000000e+02]
 [ 0.00000000e+00 -1.17187500e+00  0.00000000e+00  1.85300003e+02]
 [ 0.00000000e+00  0.00000000e+00  5.00000000e+00 -1.43419995e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [1510]:
from typing import Union

# define an utility for Nifty file saving
def save_nifty(data: Union[np.ndarray, torch.Tensor], affine, path, dtype=np.float32):
    if isinstance(data, torch.Tensor):
        data = data.detach().cpu().numpy()
    # set fixed type float32
    data = data.astype(dtype)
    nib.save(nib.Nifti1Image(data, affine), path)

def save_nifty_binary(data: Union[np.ndarray, torch.Tensor], affine, path):
    return save_nifty(data, affine, path, dtype=np.uint8)


# define an utility for annoying nnunetv2 preprocessing
def nnunetv2_default_preprocessing(ct_img_path, 
                                   predictor, 
                                   dataset_json_path,
                                   other_volumes: Union[np.ndarray, None] = None) -> np.ndarray:
    """
    Preprocesses the CT image and other volumes using nnunetv2's default preprocessing
    pipeline. This function reads the CT image, applies the preprocessor, and returns
    the preprocessed image.
    """
    plans_manager = predictor.plans_manager
    configuration_manager = predictor.configuration_manager
    
    preprocessor = configuration_manager.preprocessor_class(verbose=False)
    rw = plans_manager.image_reader_writer_class()
    if callable(rw) and not hasattr(rw, "read_images"):
        rw = rw()
    img_np, img_props = rw.read_images([str(ct_img_path)])
    
    preprocessed, other_volumes_preprocessed, _ = preprocessor.run_case_npy(
        img_np, seg=other_volumes, properties=img_props,
        plans_manager=plans_manager,
        configuration_manager=configuration_manager,
        dataset_json=dataset_json_path
    )
    if other_volumes:
        return preprocessed, other_volumes_preprocessed
    return preprocessed

## Re-align CT scan with its own organ segmentation mask

### apparently, this was solved in the remaining volumes

In [1511]:
"""import SimpleITK as sitk

# Load CT and misaligned organ mask
ct = sitk.ReadImage(ct_img_path, sitk.sitkFloat32)
organ_map = sitk.ReadImage(organ_map_path, sitk.sitkUInt8)

# Resample organ mask to match CT space
resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(ct)
resampler.SetInterpolator(sitk.sitkNearestNeighbor)
organ_resampled = resampler.Execute(organ_map)

# Save aligned output
sitk.WriteImage(organ_resampled, "organ_map_resampled_to_ct.nii.gz")"""

'import SimpleITK as sitk\n\n# Load CT and misaligned organ mask\nct = sitk.ReadImage(ct_img_path, sitk.sitkFloat32)\norgan_map = sitk.ReadImage(organ_map_path, sitk.sitkUInt8)\n\n# Resample organ mask to match CT space\nresampler = sitk.ResampleImageFilter()\nresampler.SetReferenceImage(ct)\nresampler.SetInterpolator(sitk.sitkNearestNeighbor)\norgan_resampled = resampler.Execute(organ_map)\n\n# Save aligned output\nsitk.WriteImage(organ_resampled, "organ_map_resampled_to_ct.nii.gz")'

In [1512]:
"""#organ_map_path = join(nnUNet_raw, "organ_map_resampled_to_ct.nii.gz")
organ_map_path = "organ_map_resampled_to_ct.nii.gz"
ct_img = nib.load(ct_img_path)
organ_map = nib.load(organ_map_path)
print("CT shape:", ct_img.shape)
print("Organ shape:", organ_map.shape)
print("Spacing:", ct_img.header.get_zooms())
print("Organ spacing:", organ_map.header.get_zooms())"""

'#organ_map_path = join(nnUNet_raw, "organ_map_resampled_to_ct.nii.gz")\norgan_map_path = "organ_map_resampled_to_ct.nii.gz"\nct_img = nib.load(ct_img_path)\norgan_map = nib.load(organ_map_path)\nprint("CT shape:", ct_img.shape)\nprint("Organ shape:", organ_map.shape)\nprint("Spacing:", ct_img.header.get_zooms())\nprint("Organ spacing:", organ_map.header.get_zooms())'

In [1513]:
# model directory; note that this is readonly in Kaggle environment
if IN_KAGGLE:
    model_dir = join(nnUNet_results, 'Dataset003_AUTOMI_CTVLNF_NEWGL_results/nnUNetTrainer__nnUNetPlans__3d_fullres')
else:
    model_dir = join(nnUNet_results, 'nnUNetTrainer__nnUNetPlans__3d_fullres')

## Utility to export logits to a visualizable segmentation

In [1514]:
import numpy as np
import torch
import os
from typing import Union
from pathlib import Path
from nnunetv2.configuration import default_num_processes
from nnunetv2.inference.export_prediction import export_prediction_from_logits

def export_logits_to_nifty_segmentation(
    predictor,
    volume_file: Path,
    model_dir: str,
    logits: Union[str, np.ndarray, torch.Tensor],
    npz_dir: str | None,
    output_dir: str = "",
    fold: int = 0,
    save_probs: bool = False,
    from_file: bool = True
):
    """
    Converts a saved .npz logits file into a native-space NIfTI segmentation using nnU-Net's helper.

    Args:
        predictor: An instantiated nnU-Net predictor object with loaded plans/configs.
        volume_file: An Path object pointing to the raw image file.
        model_dir (str): Path to the nnU-Net mode1Introductionl directory containing dataset.json.
        logits (str or tensor): Base name of the .npz logits file (no extension), when from_file=True
        npz_dir (str): Directory where the .npz file is stored.
        output_dir (str): Directory where the .nii.gz segmentation will be saved.
        fold (int): The fold number used for prediction (default is 0).
        save_probs (bool): Whether to save softmax probabilities as a .npz file.
        from_file (bool). Whether to convert from a file instead of from the logits (default true)
    """
    if from_file:
        npz_logits = Path(npz_dir) / f"{logits}.npz"
        output_nii = Path(output_dir) / f"{logits}_seg.nii.gz"
        logits = np.load(npz_logits)["logits"]
    else:
        output_nii = Path(output_dir) / "exported_seg.nii.gz"

    plans_manager = predictor.plans_manager
    configuration_manager = predictor.configuration_manager
    dataset_json = Path(model_dir) / "dataset.json"

    preprocessor = configuration_manager.preprocessor_class(verbose=False)
    rw = plans_manager.image_reader_writer_class()
    if callable(rw) and not hasattr(rw, "read_images"):
        rw = rw()
    img_np, img_props = rw.read_images([str(volume_file)])

    _, _, data_props = preprocessor.run_case_npy(
        img_np, seg=None, properties=img_props,
        plans_manager=plans_manager,
        configuration_manager=configuration_manager,
        dataset_json=dataset_json
    )


    export_prediction_from_logits(
        predicted_array_or_file=logits,
        properties_dict=data_props,
        configuration_manager=configuration_manager,
        plans_manager=plans_manager,
        dataset_json_dict_or_file=str(dataset_json),
        output_file_truncated=os.path.splitext(str(output_nii))[0],
        save_probabilities=save_probs,
        num_threads_torch=default_num_processes
    )

    print(f"✅  NIfTI segmentation written → {output_nii}")

In [1515]:
"""export_logits_to_nifty_segmentation(
    predictor=predictor,
    plan=plan,
    model_dir=Path(model_dir),
    logits_filename="pred_00007",
    npz_dir="SHAP/shap_run",
    output_dir="SHAP/shap_run",
    fold=0,
    save_probs=False
)"""

'export_logits_to_nifty_segmentation(\n    predictor=predictor,\n    plan=plan,\n    model_dir=Path(model_dir),\n    logits_filename="pred_00007",\n    npz_dir="SHAP/shap_run",\n    output_dir="SHAP/shap_run",\n    fold=0,\n    save_probs=False\n)'

## We define a sliding window caching for faster multi-inference scenario, like SHAP

### Try to override the sliding_window_function

In [1516]:
from typing import Union
import time
import numpy as np
import torch
from tqdm import tqdm
from queue import Queue
from threading import Thread
from acvl_utils.cropping_and_padding.padding import pad_nd_image
from nnunetv2.utilities.helpers import empty_cache, dummy_context
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
from nnunetv2.inference.sliding_window_prediction import compute_gaussian, compute_steps_for_sliding_window

class CustomNNUNetPredictor(nnUNetPredictor):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    @torch.inference_mode()
    def predict_sliding_window_return_logits_with_caching(self, input_image: torch.Tensor,
                                                          perturbation_mask: torch.BoolTensor | None,
                                                          baseline_prediction_dict: dict) \
            -> Union[np.ndarray, torch.Tensor]:
        """
        Method predict_sliding_window_return_logits taken from official nnunetv2 documentation:
        https://github.com/MIC-DKFZ/nnUNet/blob/58a3b121a6d1846a978306f6c79a7c005b7d669b/nnunetv2/inference/predict_from_raw_data.py
        We add a perturbation_mask parameter to check each patch for the actual presence of a perturbation
        """
        # fallback to original method if perturbation_mask is None
        if perturbation_mask is None:
            return self.predict_sliding_window_return_logits(input_image)
                
        assert isinstance(input_image, torch.Tensor)
        self.network = self.network.to(self.device)
        self.network.eval()

        # Autocast can be annoying
        # If the device_type is 'cpu' then it's slow as heck on some CPUs (no auto bfloat16 support detection)
        # and needs to be disabled.
        # If the device_type is 'mps' then it will complain that mps is not implemented, even if enabled=False
        # is set. Whyyyyyyy. (this is why we don't make use of enabled=False)
        # So autocast will only be active if we have a cuda device.
        with torch.autocast(self.device.type, enabled=True) if self.device.type == 'cuda' else dummy_context():
            assert input_image.ndim == 4, 'input_image must be a 4D np.ndarray or torch.Tensor (c, x, y, z)'

            
            if self.verbose:
                print(f'Input shape: {input_image.shape}')
                print("step_size:", self.tile_step_size)
                print("mirror_axes:", self.allowed_mirroring_axes if self.use_mirroring else None)
                print(f'Perturbation mask shape: {perturbation_mask.shape}')

            # DEBUG
            """# save as nifti the input image and the perturbation map
            affine = nib.load("supervoxel_map.nii.gz").affine
            save_nifty(torch.permute(input_image[0], (2,1,0)), affine, "input_image_debug.nii.gz")
            save_nifty(torch.permute(perturbation_mask[0], (2,1,0)), affine, "perturbation_mask_debug.nii.gz")"""

            # if input_image is smaller than tile_size we need to pad it to tile_size.
            data, slicer_revert_padding = pad_nd_image(input_image, self.configuration_manager.patch_size,
                                                       'constant', {'value': 0}, True,
                                                       None)

            # slicers can be applied to both perturbed volume and 
            slicers = self._internal_get_sliding_window_slicers(data.shape[1:])

            if self.perform_everything_on_device and self.device != 'cpu':
                # behavior changed
                try:
                    predicted_logits = self._internal_predict_sliding_window_return_logits(
                        data, slicers, True, perturbation_mask, baseline_prediction_dict, caching=True
                    )
                except RuntimeError as e:
                    if "CUDA out of memory" in str(e):
                        print("⚠️  CUDA OOM, cambiare batch size o patch size!")
                        raise
                    else:
                        # Mostra l'errore reale e aborta: niente CPU fallback
                        raise
            else:
                predicted_logits = self._internal_predict_sliding_window_return_logits(data, slicers,
                                                                                       self.perform_everything_on_device)

            empty_cache(self.device)
            # revert padding
            predicted_logits = predicted_logits[(slice(None), *slicer_revert_padding[1:])]
        return predicted_logits
                

    def _slice_key(self, slicer_tuple):
        # make slicer object hashable to use it for cache lookup
        return tuple((s.start, s.stop, s.step) for s in slicer_tuple)


    @torch.inference_mode()
    def _internal_predict_sliding_window_return_logits(self,
                                                       data: torch.Tensor,
                                                       slicers,
                                                       do_on_device: bool = True,
                                                       perturbation_mask: torch.BoolTensor | None = None,
                                                       baseline_prediction_dict: dict | None = None,
                                                       caching: bool = False,
                                                       ):
        """
        Modified to manage the caching of patches
        """
        predicted_logits = n_predictions = prediction = gaussian = workon = None
        results_device = self.device if do_on_device else torch.device('cpu')
        if next(self.network.parameters()).device != results_device:
            self.network = self.network.to(results_device)

        def producer(d, slh, q):
            for s in slh:
                q.put((torch.clone(d[s][None], memory_format=torch.contiguous_format).to(results_device), s))
            q.put('end')

        try:
            empty_cache(self.device)

            # move data to device
            if self.verbose:
                print(f'move image to device {results_device}')
            data = data.to(results_device)
            queue = Queue(maxsize=2)
            t = Thread(target=producer, args=(data, slicers, queue))
            t.start()

            # preallocate arrays
            if self.verbose:
                print(f'preallocating results arrays on device {results_device}')
            predicted_logits = torch.zeros((self.label_manager.num_segmentation_heads, *data.shape[1:]),
                                           dtype=torch.half,
                                           device=results_device)
            n_predictions = torch.zeros(data.shape[1:], dtype=torch.half, device=results_device)

            if self.use_gaussian:
                gaussian = compute_gaussian(tuple(self.configuration_manager.patch_size), sigma_scale=1. / 8,
                                            value_scaling_factor=10,
                                            device=results_device)
            else:
                gaussian = 1

        

            if not self.allow_tqdm and self.verbose:
                print(f'running prediction: {len(slicers)} steps')

            # Before starting queue processing:
            start_time = time.time()
            
            with tqdm(desc=None, total=len(slicers), disable=not self.allow_tqdm) as pbar:
                cache_hits = 0
                cache_misses = 0
                while True:
                    item = queue.get()
                    if item == 'end':
                        queue.task_done()
                        break
                    workon, sl = item
                    try:
                        if caching and not self.check_overlapping(sl, perturbation_mask):
                            prediction = baseline_prediction_dict[self._slice_key(sl)].to(results_device)
                            cache_hits += 1
                        else:
                            prediction = self._internal_maybe_mirror_and_predict(workon)[0].to(results_device)
                            cache_misses += 1
                    except Exception as e:
                        raise RuntimeError("Errore nella predizione del patch") from e
            
                    assert prediction.device == predicted_logits.device
            
                    if self.use_gaussian:
                        prediction *= gaussian
                    predicted_logits[sl] += prediction
                    n_predictions[sl[1:]] += gaussian
            
                    del prediction, workon
                    queue.task_done()
                    pbar.set_postfix(
                        cache=f"{cache_hits}",
                        mem=f"{torch.cuda.memory_allocated()/1e9:.2f} GB"
                    )
                    pbar.update(1)
            
            queue.join()
            
            if caching:
                # Final metrics
                hit_ratio = (cache_hits / len(slicers) * 100) if len(slicers) > 0 else 0.0
                elapsed_time = time.time() - start_time
            
                if self.verbose and not self.allow_tqdm:
                    print(f"Cache hits: {cache_hits}\\{len(slicers)}")
                    print(f"Inference time: {elapsed_time:.2f} sec")

                # Log to W&B (only if you are logged and there is a run)
                if WANDB:
                    wandb.log({
                        "cache_hit_ratio_percent": hit_ratio,
                        "inference_time_sec": elapsed_time
                    })


            # predicted_logits /= n_predictions
            torch.div(predicted_logits, n_predictions, out=predicted_logits)
            # check for infs
            if torch.any(torch.isinf(predicted_logits)):
                raise RuntimeError('Encountered inf in predicted array. Aborting... If this problem persists, '
                                   'reduce value_scaling_factor in compute_gaussian or increase the dtype of '
                                   'predicted_logits to fp32')
        except Exception as e:
            del predicted_logits, n_predictions, prediction, gaussian, workon
            empty_cache(self.device)
            empty_cache(results_device)
            raise e
        return predicted_logits
  


    def get_output_dictionary_sliding_window(self, data: torch.Tensor, slicers,
                                            do_on_device: bool = True,
                                            ) -> torch.Tensor:
        """
        # create a dictionary that associates the output of the inference, to each slicer of the sliding window module
        # this way we can set ready for cache the output for the untouched patches.
        """
        
        dictionary = dict()
        prediction = workon = None
        results_device = self.device if do_on_device else torch.device('cpu')
        if next(self.network.parameters()).device != results_device:
            self.network = self.network.to(results_device)

        def producer(d, slh, q):
            for s in slh:
                #tqdm.write(f"put patch {s} on queue")    # dentro producer
                q.put((torch.clone(d[s][None], memory_format=torch.contiguous_format).to(self.device), s))
            q.put('end')

        try:
            empty_cache(self.device)

            # move data and network to device
            if self.verbose:
                print(f'move image and model to device {results_device}')

            self.network = self.network.to(results_device)
            data = data.to(results_device)
            queue = Queue(maxsize=2)
            t = Thread(target=producer, args=(data, slicers, queue))
            t.start()

            if not self.allow_tqdm and self.verbose:
                print(f'running prediction: {len(slicers)} steps')

            with tqdm(desc=None, total=len(slicers), disable=not self.allow_tqdm) as pbar:
                while True:
                    item = queue.get()
                    if item == 'end':
                        queue.task_done()
                        break
                    workon, sl = item
                    pred_gpu = self._internal_maybe_mirror_and_predict(workon)[0].to(results_device)

                    pred_cpu = pred_gpu.cpu()
                    # save prediction in the dictionary
                    dictionary[self._slice_key(sl)] = pred_cpu
                    # immediately free gpu memory
                    del pred_gpu
                    
                    queue.task_done()
                    pbar.update()
            queue.join()

        except Exception as e:
            del workon#, prediction
            empty_cache(self.device)
            empty_cache(results_device)
            raise e
        return dictionary



    def check_overlapping(self, slicer, perturbation_mask: torch.BoolTensor) -> bool:
        """
        Restituisce True se la patch definita da `slicer`
        contiene almeno un voxel perturbato.
    
        Parameters
        ----------
        slicer : tuple
            Quello prodotto da `_internal_get_sliding_window_slicers`,
            cioè (slice(None), slice(x0,x1), slice(y0,y1), slice(z0,z1)).
        perturbation_mask : torch.BoolTensor
            Maschera (C, X, Y, Z) con True nei voxel da perturbare
            (di solito C==1 o replicata sui canali).
    
        Returns
        -------
        bool
            True ↔ almeno un voxel True nella patch.out
        """
        # NB: il primo elemento del tuple è sempre slice(None) (canali).
        #     Lo manteniamo: non ha overhead e semplifica.
        return perturbation_mask[slicer].any().item()

    from datetime import datetime
    
    def log_cache_stats(log_file: str,
                        total_patches: int,
                        cache_hits: int,
                        cache_misses: int,
                        extra_info: dict | None = None):
        """
        Append caching statistics to a JSON log file.
    
        Args:
            log_file (str): Path to the JSON file.
            total_patches (int): Total number of patches processed.
            cache_hits (int): Number of cache hits.
            cache_misses (int): Number of cache misses.
            extra_info (dict, optional): Additional run-specific information to log.
        """
        hit_ratio = (cache_hits / total_patches * 100) if total_patches > 0 else 0.0
        log_entry = {
            "timestamp": datetime.now().isoformat(timespec="seconds"),
            "total_patches": total_patches,
            "cache_hits": cache_hits,
            "cache_misses": cache_misses,
            "hit_ratio_percent": round(hit_ratio, 2)
        }
    
        if extra_info:
            log_entry.update(extra_info)
    
        # Load existing logs
        if os.path.exists(log_file):
            try:
                with open(log_file, "r", encoding="utf-8") as f:
                    logs = json.load(f)
                if not isinstance(logs, list):
                    logs = []
            except json.JSONDecodeError:
                logs = []
        else:
            logs = []
    
        logs.append(log_entry)
    
        with open(log_file, "w", encoding="utf-8") as f:
            json.dump(logs, f, indent=2)
    
        print(f"[CACHE] Logged stats to '{log_file}'")

## Try Captum's kernel SHAP on the organ mask

### first derive a customized class from Captum library, to use sliding window caching

## Try to customize KernelShap as a "sibling", so let's inherit the parent, LimeBase
that's because we need to override (to-and-from)/interpret_rep_transform methods used to map the (1,M) binary mask vector with the perturbed volume AND the perturbation mask we need for caching

In [1517]:
#!/usr/bin/env python3

# pyre-strict
import inspect
import math
import typing
import warnings
from collections.abc import Iterator
from typing import Any, Callable, cast, List, Literal, Optional, Tuple, Union

import torch
from captum._utils.common import (
    _expand_additional_forward_args,
    _expand_target,
    _flatten_tensor_or_tuple,
    _format_output,
    _format_tensor_into_tuples,
    _get_max_feature_index,
    _is_tuple,
    _reduce_list,
    _run_forward,
)
from captum._utils.models.linear_model import SkLearnLasso
from captum._utils.models.model import Model
from captum._utils.progress import progress
from captum._utils.typing import BaselineType, TargetType, TensorOrTupleOfTensorsGeneric
from captum.attr._utils.attribution import PerturbationAttribution
from captum.attr._utils.batching import _batch_example_iterator
from captum.attr._utils.common import (
    _construct_default_feature_mask,
    _format_input_baseline,
)
from captum.log import log_usage
from torch import Tensor, BoolTensor
from torch.nn import CosineSimilarity
from torch.utils.data import DataLoader, TensorDataset


class LimeBaseWithCustomArgumentToForwardFunc(PerturbationAttribution):
    r"""
    Here we create a modification of Lime class from Captum Library (https://captum.ai/api/_modules/captum/attr/_core/lime.html)
    """

    def __init__(
        self,
        forward_func: Callable[..., Tensor],
        interpretable_model: Model,
        similarity_func: Callable[
            ...,
            Union[float, Tensor],
        ],
        perturb_func: Callable[..., object],
        perturb_interpretable_space: bool,
        from_interp_rep_transform: Optional[
            Callable[..., Union[Tensor, Tuple[Tensor, ...]]]
        ],
        to_interp_rep_transform: Optional[Callable[..., Tensor]],
    ) -> None:
        r"""

        Args:


            forward_func (Callable): The forward function of the model or any
                    modification of it. If a batch is provided as input for
                    attribution, it is expected that forward_func returns a scalar
                    representing the entire batch.
            interpretable_model (Model): Model object to train interpretable model.
                    A Model object provides a `fit` method to train the model,
                    given a dataloader, with batches containing three tensors:

                    - interpretable_inputs: Tensor
                      [2D num_samples x num_interp_features],
                    - expected_outputs: Tensor [1D num_samples],
                    - weights: Tensor [1D num_samples]

                    The model object must also provide a `representation` method to
                    access the appropriate coefficients or representation of the
                    interpretable model after fitting.
                    Some predefined interpretable linear models are provided in
                    captum._utils.models.linear_model including wrappers around
                    SkLearn linear models as well as SGD-based PyTorch linear
                    models.

                    Note that calling fit multiple times should retrain the
                    interpretable model, each attribution call reuses
                    the same given interpretable model object.
            similarity_func (Callable): Function which takes a single sample
                    along with its corresponding interpretable representation
                    and returns the weight of the interpretable sample for
                    training interpretable model. Weight is generally
                    determined based on similarity to the original input.
                    The original paper refers to this as a similarity kernel.

                    The expected signature of this callable is:

                    >>> similarity_func(
                    >>>    original_input: Tensor or tuple[Tensor, ...],
                    >>>    perturbed_input: Tensor or tuple[Tensor, ...],
                    >>>    perturbed_interpretable_input:
                    >>>        Tensor [2D 1 x num_interp_features],
                    >>>    **kwargs: Any
                    >>> ) -> float or Tensor containing float scalar

                    perturbed_input and original_input will be the same type and
                    contain tensors of the same shape (regardless of whether or not
                    the sampling function returns inputs in the interpretable
                    space). original_input is the same as the input provided
                    when calling attribute.

                    All kwargs passed to the attribute method are
                    provided as keyword arguments (kwargs) to this callable.
            perturb_func (Callable): Function which returns a single
                    sampled input, generally a perturbation of the original
                    input, which is used to train the interpretable surrogate
                    model. Function can return samples in either
                    the original input space (matching type and tensor shapes
                    of original input) or in the interpretable input space,
                    which is a vector containing the intepretable features.
                    Alternatively, this function can return a generator
                    yielding samples to train the interpretable surrogate
                    model, and n_samples perturbations will be sampled
                    from this generator.

                    The expected signature of this callable is:

                    >>> perturb_func(
                    >>>    original_input: Tensor or tuple[Tensor, ...],
                    >>>    **kwargs: Any
                    >>> ) -> Tensor, tuple[Tensor, ...], or
                    >>>    generator yielding tensor or tuple[Tensor, ...]

                    All kwargs passed to the attribute method are
                    provided as keyword arguments (kwargs) to this callable.

                    Returned sampled input should match the input type (Tensor
                    or Tuple of Tensor and corresponding shapes) if
                    perturb_interpretable_space = False. If
                    perturb_interpretable_space = True, the return type should
                    be a single tensor of shape 1 x num_interp_features,
                    corresponding to the representation of the
                    sample to train the interpretable model.

                    All kwargs passed to the attribute method are
                    provided as keyword arguments (kwargs) to this callable.
            perturb_interpretable_space (bool): Indicates whether
                    perturb_func returns a sample in the interpretable space
                    (tensor of shape 1 x num_interp_features) or a sample
                    in the original space, matching the format of the original
                    input. Once sampled, inputs can be converted to / from
                    the interpretable representation with either
                    to_interp_rep_transform or from_interp_rep_transform.
            from_interp_rep_transform (Callable): Function which takes a
                    single sampled interpretable representation (tensor
                    of shape 1 x num_interp_features) and returns
                    the corresponding representation in the input space
                    (matching shapes of original input to attribute).

                    This argument is necessary if perturb_interpretable_space
                    is True, otherwise None can be provided for this argument.

                    The expected signature of this callable is:

                    >>> from_interp_rep_transform(
                    >>>    curr_sample: Tensor [2D 1 x num_interp_features]
                    >>>    original_input: Tensor or Tuple of Tensors,
                    >>>    **kwargs: Any
                    >>> ) -> Tensor or tuple[Tensor, ...]

                    Returned sampled input should match the type of original_input
                    and corresponding tensor shapes.

                    All kwargs passed to the attribute method are
                    provided as keyword arguments (kwargs) to this callable.

            to_interp_rep_transform (Callable): Function which takes a
                    sample in the original input space and converts to
                    its interpretable representation (tensor
                    of shape 1 x num_interp_features).

                    This argument is necessary if perturb_interpretable_space
                    is False, otherwise None can be provided for this argument.

                    The expected signature of this callable is:

                    >>> to_interp_rep_transform(
                    >>>    curr_sample: Tensor or Tuple of Tensors,
                    >>>    original_input: Tensor or Tuple of Tensors,
                    >>>    **kwargs: Any
                    >>> ) -> Tensor [2D 1 x num_interp_features]

                    curr_sample will match the type of original_input
                    and corresponding tensor shapes.

                    All kwargs passed to the attribute method are
                    provided as keyword arguments (kwargs) to this callable.
        """
        PerturbationAttribution.__init__(self, forward_func)
        self.interpretable_model = interpretable_model
        self.similarity_func = similarity_func
        self.perturb_func = perturb_func
        self.perturb_interpretable_space = perturb_interpretable_space
        self.from_interp_rep_transform = from_interp_rep_transform
        self.to_interp_rep_transform = to_interp_rep_transform

        if self.perturb_interpretable_space:
            assert (
                self.from_interp_rep_transform is not None
            ), "Must provide transform from interpretable space to original input space"
            " when sampling from interpretable space."
        else:
            assert (
                self.to_interp_rep_transform is not None
            ), "Must provide transform from original input space to interpretable space"

    @log_usage(part_of_slo=True)
    @torch.no_grad()
    def attribute(
        self,
        inputs: TensorOrTupleOfTensorsGeneric,
        target: TargetType = None,
        additional_forward_args: Optional[Tuple[object, ...]] = None,
        n_samples: int = 50,
        perturbations_per_eval: int = 1,
        show_progress: bool = False,
        # --- MONITOR CONVERGENCE QUALITY
        monitor_log_path: str | None = None,
        monitor_convergence_step: int | None = 20,
        monitor_local_accuracy_step: int | None = 50,
        **kwargs: object,
    ) -> Tensor:
        r"""
        This method attributes the output of the model with given target index
        (in case it is provided, otherwise it assumes that output is a
        scalar) to the inputs of the model using the approach described above.
        It trains an interpretable model and returns a representation of the
        interpretable model.

        It is recommended to only provide a single example as input (tensors
        with first dimension or batch size = 1). This is because LIME is generally
        used for sample-based interpretability, training a separate interpretable
        model to explain a model's prediction on each individual example.

        A batch of inputs can be provided as inputs only if forward_func
        returns a single value per batch (e.g. loss).
        The interpretable feature representation should still have shape
        1 x num_interp_features, corresponding to the interpretable
        representation for the full batch, and perturbations_per_eval
        must be set to 1.

        Args:

            inputs (Tensor or tuple[Tensor, ...]): Input for which LIME
                        is computed. If forward_func takes a single
                        tensor as input, a single input tensor should be provided.
                        If forward_func takes multiple tensors as input, a tuple
                        of the input tensors should be provided. It is assumed
                        that for all given input tensors, dimension 0 corresponds
                        to the number of examples, and if multiple input tensors
                        are provided, the examples must be aligned appropriately.
            target (int, tuple, Tensor, or list, optional): Output indices for
                        which surrogate model is trained
                        (for classification cases,
                        this is usually the target class).
                        If the network returns a scalar value per example,
                        no target index is necessary.
                        For general 2D outputs, targets can be either:

                        - a single integer or a tensor containing a single
                          integer, which is applied to all input examples

                        - a list of integers or a 1D tensor, with length matching
                          the number of examples in inputs (dim 0). Each integer
                          is applied as the target for the corresponding example.

                        For outputs w            except --------ith > 2 dimensions, targets can be either:

                        - A single tuple, which contains #output_dims - 1
                          elements. This target index is applied to all examples.

                        - A list of tuples with length equal to the number of
                          examples in inputs (dim 0), and each tuple containing
                          #output_dims - 1 elements. Each tuple is applied as the
                          target for the corresponding example.

                        Default: None
            additional_forward_args (Any, optional): If the forward function
                        requires additional arguments other than the inputs for
                        which attributions should not be computed, this argument
                        can be provided. It must be either a single additional
                        argument of a Tensor or arbitrary (non-tuple) type or a
                        tuple containing multiple additional arguments including
                        tensors or any arbitrary python types. These arguments
                        are provided to forward_func in order following the
                        arguments in inputs.
                        For a tensor, the first dimension of the tensor must
                        correspond to the number of examples. For all other types,
                        the given argument is used for all forward evaluations.
                        Note that attributions are not computed with respect
                        to these arguments.
                        Default: None
            n_samples (int, optional): The number of samples of the original
                        model used to train the surrogate interpretable model.
                        Default: `50` if `n_samples` is not provided.
            perturbations_per_eval (int, optional): Allows multiple samples
                        to be processed simultaneously in one call to forward_fn.
                        Each forward pass will contain a maximum of
                        perturbations_per_eval * #examples samples.
                        For DataParallel models, each batch is split among the
                        available devices, so evaluations on each available
                        device contain at most
                        (perturbations_per_eval * #examples) / num_devices
                        samples.
                        If the forward function returns a single scalar per batch,
                        perturbations_per_eval must be set to 1.
                        Default: 1
            show_progress (bool, optional): Displays the progress of computation.
                        It will try to use tqdm if available for advanced features
                        (e.g. time estimation). Otherwise, it will fallback to
                        a simple output of progress.
                        Default: False
            monitor_log_path (str, optional): Path to the log file for monitoring convergence.
                        if None, no monitoring is performed.
                        Default: None
            monitor_convergence_step (int, optional): Number of iterations over which
                        the difference among two attribution is computerd.
                        Default: 20
            monitor_local_accuracy_step (int, optional): Number of iterations over which
                        the local accuracy of an attribution is computerd.
                        Default: 50
            **kwargs (Any, optional): Any additional arguments necessary for
                        sampling and transformation functions (provided to
                        constructor).
                        Default: None

        Returns:
            **interpretable model representation**:
            - **interpretable model representation** (*Any*):
                    A representation of the interpretable model trained. The return
                    type matches the return type of train_interpretable_model_func.
                    For example, this could contain coefficients of a
                    linear surrogate model.

        Examples::

            >>> # SimpleClassifier takes a single input tensor of
            >>> # float features with size N x 5,
            >>> # and returns an Nx3 tensor of class probabilities.
            >>> net = SimpleClassifier()
            >>>
            >>> # We will train an interpretable model with the same
            >>> # features by simply sampling with added Gaussian noise
            >>> # to the inputs and training a model to predict the
            >>> # score of the target class.
            >>>
            >>> # For interpretable model training, we will use sklearn
            >>> # linear model in this example. We have provided wrappers
            >>> # around sklearn linear models to fit the Model interface.
            >>> # Any arguments provided to the sklearn constructor can also
            >>> # be provided to the wrapper, e.g.:
            >>> # SkLearnLinearModel("linear_model.Ridge", alpha=2.0)
            >>> from captum._utils.models.linear_model import SkLearnLinearModel
            >>>
            >>>
            >>> # Define similarity kernel (exponential kernel based on L2 norm)
            >>> def similarity_kernel(
            >>>     original_input: Tensor,
            >>>     perturbed_input: Tensor,
            >>>     perturbed_interpretable_input: Tensor,
            >>>     **kwargs)->Tensor:
            >>>         # kernel_width will be provided to attribute as a kwarg
            >>>         kernel_width = kwargs["kernel_width"]
            >>>         l2_dist = torch.norm(original_input - perturbed_input)
            >>>         return torch.exp(- (l2_dist**2) / (kernel_width**2))
            >>>
            >>>
            >>> # Define sampling function
            >>> # This function samples in original input space
            >>> def perturb_func(
            >>>     original_input: Tensor,
            >>>     **kwargs)->Tensor:
            >>>         return original_input + torch.randn_like(original_input)
            >>>
            >>> # For this example, we are setting the interpretable input to
            >>> # match the model input, so the to_interp_rep_transform
            >>> # function simply returns the input. In most cases, the interpretable
            >>> # input will be different and may have a smaller feature set, so
            >>> # an appropriate transformation function should be provided.
            >>>
            >>> def to_interp_transform(curr_sample, original_inp,
            >>>                                      **kwargs):
            >>>     return curr_sample\
            >>>
            >>> # Generating random input with size 1 x 5
            >>> input = torch.randn(1, 5)
            >>> # Defining LimeBase interpreter
            >>> lime_attr = LimeBase(net,
                                     SkLearnLinearModel("linear_model.Ridge"),
                                     similarity_func=similarity_kernel,
                                     perturb_func=perturb_func,
                                     perturb_interpretable_space=False,
                                     from_interp_rep_transform=None,
                                     to_interp_rep_transform=to_interp_transform)
            >>> # Computes interpretable model, returning coefficients of linear
            >>> # model.
            >>> attr_coefs = lime_attr.attribute(input, target=1, kernel_width=1.1)
        """
        inp_tensor = cast(Tensor, inputs) if isinstance(inputs, Tensor) else inputs[0]
        device = inp_tensor.device
        
        # --------------------------------------------------------------------- #
        # 1.  Lists that grow while we sample                                   #
        # --------------------------------------------------------------------- #
        interpretable_inps, similarities, outputs = [], [], []
        
        curr_model_inputs = []
        expanded_additional_args = None
        expanded_target = None
        gen_perturb_func = self._get_perturb_generator_func(inputs, **kwargs)

        # --------------------------------------------------------------------- #
        # 2.  Monitoring initialisation                                         #
        # --------------------------------------------------------------------- #
        MONITOR = monitor_log_path is not None
        if MONITOR:
            beta_prev = None
            k_monitor_conv   = monitor_convergence_step
            k_monitor_delta  = monitor_local_accuracy_step
            logf = open(monitor_log_path, "a")
        
        if show_progress:
            attr_progress = progress(
                total=math.ceil(n_samples / perturbations_per_eval),
                desc=f"{self.get_name()} attribution",
            )
            attr_progress.update(0)
        
        feature_mask = kwargs["feature_mask"]
        batch_count = 0

        # --------------------------------------------------------------------- #
        # 3.  Main sampling loop                                                #
        # --------------------------------------------------------------------- #
        for _ in range(n_samples):
            try:
                interpretable_inp, curr_model_input = gen_perturb_func()
                perturbation_mask = self._get_perturbation_mask(
                    interpretable_inp, curr_model_input, feature_mask, kwargs["attribute_background"]
                )
            except StopIteration:
                warnings.warn(
                    "Generator completed prior to given n_samples iterations!",
                    stacklevel=1,
                )
                break
            except Exception:
                print("error in the perturbation mask generation")
                raise
        
            # ------------ Build forward args with mask ------------------------ #
            if additional_forward_args is None:
                additional_forward_args_with_mask = (perturbation_mask,)
            elif isinstance(additional_forward_args, tuple):
                additional_forward_args_with_mask = additional_forward_args + (perturbation_mask,)
            else:
                additional_forward_args_with_mask = (additional_forward_args, perturbation_mask)

            # ------------ Book-keeping per sample ----------------------------- #
            batch_count += 1
            interpretable_inps.append(interpretable_inp)
            curr_model_inputs.append(curr_model_input)
        
            curr_sim = self.similarity_func(inputs, curr_model_input, interpretable_inp, **kwargs)
            similarities.append(
                curr_sim.flatten()
                if isinstance(curr_sim, Tensor)
                else torch.tensor([curr_sim], device=device)
            )

            # ------------ When we have one evaluation batch ready ------------- #
            if len(curr_model_inputs) == perturbations_per_eval:
                expanded_additional_args = _expand_additional_forward_args(
                    additional_forward_args_with_mask, len(curr_model_inputs)
                )
                if expanded_target is None:
                    expanded_target = _expand_target(target, len(curr_model_inputs))
        
                model_out = self._evaluate_batch(
                    curr_model_inputs,
                    expanded_target,
                    expanded_additional_args,
                    device,
                )
                if show_progress:
                    attr_progress.update()
                outputs.append(model_out)
        
                curr_model_inputs = []

                # =============================================================== #
                # === MONITORING: re-fit & log when a checkpoint is reached ===== #
                # =============================================================== #
                if MONITOR and (
                    len(interpretable_inps) % k_monitor_conv == 0
                    or len(interpretable_inps) % k_monitor_delta == 0
                ):
                    ### MONITOR BEGIN
                    # build DataLoader with *all* samples so far
                    X = torch.cat(interpretable_inps).float()
                    y = (
                        torch.cat(outputs)
                        if len(outputs[0].shape) > 0
                        else torch.stack(outputs)
                    ).float()
                    w = torch.cat(similarities).float()
        
                    dl_mon = DataLoader(
                        TensorDataset(X, y, w), batch_size=len(X)
                    )
        
                    # one API call → fast enough for ≤ few 100 samples
                    self.interpretable_model.fit(dl_mon)
        
                    # ---------- obtain coefficients as a clean 1-D tensor -----------------
                    rep = self.interpretable_model.representation()        # may be Tensor / np / list
                    
                    beta_cur = (
                        rep.flatten().to("cpu")                         # if already a Tensor
                        if isinstance(rep, torch.Tensor)
                        else torch.as_tensor(rep, dtype=torch.float32, device="cpu").flatten() # keep these small vectors in CPU
                    )
                    phi0, phis = beta_cur[0].item(), beta_cur[1:]          # scalar + 1-D tensor
                    # ----------------------------------------------------------------------
                    
                    # ---- Convergence distance -------------------------------------------
                    if len(interpretable_inps) % k_monitor_conv == 0 and beta_prev is not None:
                        dist = torch.norm(beta_cur - beta_prev, p=1).item()
                        logf.write(json.dumps({
                            "iter": len(interpretable_inps),
                            "conv_dist_L1": dist
                        }) + "\n")

        
                    # ---- Local-accuracy residual ------------------------------ #
                    if len(interpretable_inps) % k_monitor_delta == 0:
                        # take the output of the unperturbed input (ASSUMES first sample is unperturbed, only true for KernelSHAP)
                        model_fwd_original = outputs[0]
                        fx = model_fwd_original.item() if torch.is_tensor(model_fwd_original) else model_fwd_original
                        delta = abs(fx - (phi0 + sum(phis))).item()
                        logf.write(json.dumps({
                            "iter": len(interpretable_inps),
                            "delta_shap": delta
                        }) + "\n")

                    # flush the log file
                    logf.flush()

                    """# DEBUG store whole dataset each 10 iterations
                    import pickle
                    if len(interpretable_inps) % 10 == 0:
                        with open('dataset-autosave.pkl', 'wb') as file:
                            pickle.dump(dl_mon, file)"""

                    beta_prev = beta_cur.detach().cpu()
                    ### MONITOR END


        # --------------------------------------------------------------------- #
        # 4.  Flush any leftover mini-batch                                     #
        # --------------------------------------------------------------------- #
        if len(curr_model_inputs) > 0:
            expanded_additional_args = _expand_additional_forward_args(
                additional_forward_args_with_mask, len(curr_model_inputs)
            )
            expanded_target = _expand_target(target, len(curr_model_inputs))
        
            model_out = self._evaluate_batch(
                curr_model_inputs,
                expanded_target,
                expanded_additional_args,
                device,
            )
            if show_progress:
                attr_progress.update()
            outputs.append(model_out)
        
        if show_progress:
            attr_progress.close()

        # --------------------------------------------------------------------- #
        # 5.  Final fit on *all* samples                                        #
        # --------------------------------------------------------------------- #
        combined_interp_inps = torch.cat(interpretable_inps).float()
        combined_outputs = (
            torch.cat(outputs) if len(outputs[0].shape) > 0 else torch.stack(outputs)
        ).float()
        combined_sim = (
            torch.cat(similarities)
            if len(similarities[0].shape) > 0
            else torch.stack(similarities)
        ).float()
        
        self.dataset = TensorDataset(combined_interp_inps, combined_outputs, combined_sim)
        self.interpretable_model.fit(DataLoader(self.dataset, batch_size=batch_count))
        
        if MONITOR:
            logf.close()
        
        return self.interpretable_model.representation()



    """    def _get_perturbation_mask(
        self,
        interpretable_input: torch.Tensor,       # shape = (B, M)
        original_inputs: TensorOrTupleOfTensorsGeneric, # shape = (B, C, D, H, W) or tuple thereof
        feature_mask,
    ) -> Union[torch.BoolTensor, Tuple[torch.BoolTensor, ...]]:
        
        Build a Boolean mask of shape (B, *input_dims) indicating which
        elements should be perturbed (True) vs. left untouched (False).
        
    
        # Case 1: single‐Tensor input
        if isinstance(feature_mask, torch.Tensor):
            # advanced indexing over the batch dimension
            # result has shape (B, *feature_mask.shape)
            mask = interpretable_input[:, feature_mask]
            mask = ~mask.bool()

            return mask
    
        # Case 2: multi‐input (tuple) model
        else:
            masks = []
            for fm_i in feature_mask:
                mask_i = interpretable_input[:, fm_i]  # → (B, *fm_i.shape)
                masks.append(~mask_i.bool())
            return tuple(masks)"""

    def _get_perturbation_mask(
        self,
        interpretable_input: torch.Tensor,       # (B, M) — 1 = keep original, 0 = perturb
        original_inputs: TensorOrTupleOfTensorsGeneric,  # not used here
        feature_mask,
        attribute_background: bool = True,
    ) -> Union[torch.BoolTensor, Tuple[torch.BoolTensor, ...]]:
        """
        Return a Boolean mask (B, *input_dims) where:
            True  = this voxel is perturbed → recompute sliding-window patch
            False = voxel unchanged → can skip and use cached prediction
        """
    
        # Single-tensor input space
        if isinstance(feature_mask, torch.Tensor):
            if attribute_background:
                # Map interpretable_input directly to input labels
                keep_mask = interpretable_input[:, feature_mask].bool()
                perturb_mask = ~keep_mask
            else:
                # Background (label 0) is never perturbed; feature i maps to label i+1
                B, M = interpretable_input.shape
                perturb_mask = torch.zeros((B, *feature_mask.shape),
                                           dtype=torch.bool,
                                           device=feature_mask.device)
                for i in range(M):
                    label = i + 1
                    off_batch = ~interpretable_input[:, i].bool()
                    if off_batch.any():
                        loc = (feature_mask == label)
                        perturb_mask[off_batch] |= loc
            return perturb_mask
    
        # Tuple of feature_mask tensors (multi-input)
        else:
            masks = []
            for fm in feature_mask:
                if attribute_background:
                    keep_mask = interpretable_input[:, fm].bool()
                    perturb_mask = ~keep_mask
                else:
                    B, M = interpretable_input.shape
                    perturb_mask = torch.zeros((B, *fm.shape),
                                               dtype=torch.bool,
                                               device=fm.device)
                    for i in range(M):
                        label = i + 1
                        off_batch = ~interpretable_input[:, i].bool()
                        if off_batch.any():
                            loc = (fm == label)
                            perturb_mask[off_batch] |= loc
                masks.append(perturb_mask)
            return tuple(masks)

    

    def _get_perturb_generator_func(
        self, inputs: TensorOrTupleOfTensorsGeneric, **kwargs: Any
    ) -> Callable[
        [], Tuple[TensorOrTupleOfTensorsGeneric, TensorOrTupleOfTensorsGeneric]
    ]:
        perturb_generator: Optional[Iterator[TensorOrTupleOfTensorsGeneric]]
        perturb_generator = None
        if inspect.isgeneratorfunction(self.perturb_func):
            perturb_generator = self.perturb_func(inputs, **kwargs)

        def generate_perturbation() -> (
            Tuple[TensorOrTupleOfTensorsGeneric, TensorOrTupleOfTensorsGeneric]
        ):
            if perturb_generator:
                curr_sample = next(perturb_generator)
            else:
                curr_sample = self.perturb_func(inputs, **kwargs)

            if self.perturb_interpretable_space:
                interpretable_inp = curr_sample
                curr_model_input = self.from_interp_rep_transform(  # type: ignore
                    curr_sample, inputs, **kwargs
                )
            else:
                curr_model_input = curr_sample
                interpretable_inp = self.to_interp_rep_transform(  # type: ignore
                    curr_sample, inputs, **kwargs
                )

            return interpretable_inp, curr_model_input  # type: ignore

        return generate_perturbation

    # pyre-fixme[24] Generic type `Callable` expects 2 type parameters.)
    def attribute_future(self) -> Callable:
        r"""
        This method is not implemented for LimeBase.
        """
        raise NotImplementedError(
            "LimeBase does not support attribution of future samples."
        )

    def _evaluate_batch(
        self,
        curr_model_inputs: List[TensorOrTupleOfTensorsGeneric],
        expanded_target: TargetType,
        expanded_additional_args: object,
        device: torch.device,
    ) -> Tensor:
        model_out = _run_forward(
            self.forward_func,
            #MOMENTANEAOUS---> sliding_window forward function only works with single items (no batch) --> take first
            #_reduce_list(curr_model_inputs),
            _reduce_list(curr_model_inputs)[0],
            expanded_target,
            expanded_additional_args,
        )
        if isinstance(model_out, Tensor):
            assert model_out.numel() == len(curr_model_inputs), (
                "Number of outputs is not appropriate, must return "
                "one output per perturbed input"
            )
        if isinstance(model_out, Tensor):
            return model_out.flatten()
        return torch.tensor([model_out], device=device)

    def has_convergence_delta(self) -> bool:
        return False

    @property
    def multiplies_by_inputs(self) -> bool:
        return False


# Default transformations and methods
# for Lime child implementation.


# pyre-fixme[3]: Return type must be annotated.
# pyre-fixme[2]: Parameter must be annotated.
"""def default_from_interp_rep_transform(curr_sample, original_inputs, **kwargs):
    assert (
        "feature_mask" in kwargs
    ), "Must provide feature_mask to use default interpretable representation transform"
    assert (
        "baselines" in kwargs
    ), "Must provide baselines to use default interpretable representation transform"
    feature_mask = kwargs["feature_mask"]
    if isinstance(feature_mask, Tensor):
        binary_mask = curr_sample[0][feature_mask].bool()
        input_space_transformed = (
            binary_mask.to(original_inputs.dtype) * original_inputs
            + (~binary_mask).to(original_inputs.dtype) * kwargs["baselines"]
        )
        
        return input_space_transformed
    else:
        binary_mask = tuple(
            curr_sample[0][feature_mask[j]].bool() for j in range(len(feature_mask))
        )
        input_space_transformed = tuple(
            binary_mask[j].to(original_inputs[j].dtype) * original_inputs[j]
            + (~binary_mask[j]).to(original_inputs[j].dtype) * kwargs["baselines"][j]
            for j in range(len(feature_mask))
        )
        return input_space_transformed"""

def default_from_interp_rep_transform(curr_sample, original_inputs, attribute_background=True, **kwargs):
    assert "feature_mask" in kwargs, "Must provide feature_mask to use default interpretable representation transform"
    assert "baselines" in kwargs, "Must provide baselines to use default interpretable representation transform"

    feature_mask = kwargs["feature_mask"]
    baselines    = kwargs["baselines"]

    def _build_keep_mask(labels: torch.Tensor, sample_vec: torch.Tensor, attribute_background: bool) -> torch.BoolTensor:
        """
        Returns a boolean mask 'keep':
            True  -> take from original_inputs
            False -> take from baselines
        """
        M = sample_vec.shape[0]
        sample_vec = sample_vec.bool()

        if attribute_background:
            # interpretable idx = input label directly
            valid = (labels >= 0) & (labels < M)
            idx   = torch.clamp(labels, 0, M-1)
            keep  = torch.ones_like(labels, dtype=torch.bool)
            keep[valid] = sample_vec[idx[valid]]
        else:
            # interpretable idx i -> input label i+1
            # background (0) is never perturbed
            valid = (labels >= 1) & (labels <= M)
            idx   = (labels - 1).clamp(min=0, max=M-1)
            keep  = torch.ones_like(labels, dtype=torch.bool)
            keep[valid] = sample_vec[idx[valid]]
            # all labels == 0 (background) remain True
        return keep

    if isinstance(feature_mask, torch.Tensor):
        keep_mask = _build_keep_mask(feature_mask, curr_sample[0], attribute_background)
        return keep_mask.to(original_inputs.dtype) * original_inputs + (~keep_mask).to(original_inputs.dtype) * baselines
    else:
        outputs = []
        for j in range(len(feature_mask)):
            keep_mask = _build_keep_mask(feature_mask[j], curr_sample[0], attribute_background)
            outputs.append(
                keep_mask.to(original_inputs[j].dtype) * original_inputs[j] +
                (~keep_mask).to(original_inputs[j].dtype) * baselines[j]
            )
        return tuple(outputs)


def get_exp_kernel_similarity_function(
    distance_mode: str = "cosine",
    kernel_width: float = 1.0,
) -> Callable[..., float]:
    r"""
    This method constructs an appropriate similarity function to compute
    weights for perturbed sample in LIME. Distance between the original
    and perturbed inputs is computed based on the provided distance mode,
    and the distance is passed through an exponential kernel with given
    kernel width to convert to a range between 0 and 1.

    The callable returned can be provided as the similarity_fn for
    Lime or LimeBase.

    Args:

        distance_mode (str, optional): Distance mode can be either "cosine" or
                    "euclidean" corresponding to either cosine distance
                    or Euclidean distance respectively. Distance is computed
                    by flattening the original inputs and perturbed inputs
                    (concatenating tuples of inputs if necessary) and computing
                    distances between the resulting vectors.
                    Default: "cosine"
        kernel_width (float, optional):
                    Kernel width for exponential kernel applied to distance.
                    Default: 1.0

    Returns:

        *Callable*:
        - **similarity_fn** (*Callable*):
            Similarity function. This callable can be provided as the
            similarity_fn for Lime or LimeBase.
    """

    # pyre-fixme[3]: Return type must be annotated.
    # pyre-fixme[2]: Parameter must be annotated.
    def default_exp_kernel(original_inp, perturbed_inp, __, **kwargs):
        flattened_original_inp = _flatten_tensor_or_tuple(original_inp).float()
        flattened_perturbed_inp = _flatten_tensor_or_tuple(perturbed_inp).float()
        if distance_mode == "cosine":
            cos_sim = CosineSimilarity(dim=0)
            distance = 1 - cos_sim(flattened_original_inp, flattened_perturbed_inp)
        elif distance_mode == "euclidean":
            distance = torch.norm(flattened_original_inp - flattened_perturbed_inp)
        else:
            raise ValueError("distance_mode must be either cosine or euclidean.")
        return math.exp(-1 * (distance**2) / (2 * (kernel_width**2)))

    return default_exp_kernel


def default_perturb_func(
    original_inp: TensorOrTupleOfTensorsGeneric, **kwargs: object
) -> Tensor:
    assert (
        "num_interp_features" in kwargs
    ), "Must provide num_interp_features to use default interpretable sampling function"
    if isinstance(original_inp, Tensor):
        device = original_inp.device
    else:
        device = original_inp[0].device

    probs = torch.ones(1, cast(int, kwargs["num_interp_features"])) * 0.5
    return torch.bernoulli(probs).to(device=device).long()


def construct_feature_mask(
    feature_mask: Union[None, Tensor, Tuple[Tensor, ...]],
    formatted_inputs: Tuple[Tensor, ...],
    attribute_background: bool = True
) -> Tuple[Tuple[Tensor, ...], int]:
    feature_mask_tuple: Tuple[Tensor, ...]
    if feature_mask is None:
        feature_mask_tuple, num_interp_features = _construct_default_feature_mask(
            formatted_inputs
        )
    else:
        feature_mask_tuple = _format_tensor_into_tuples(feature_mask)
        min_interp_features = int(
            min(
                torch.min(single_mask).item()
                for single_mask in feature_mask_tuple
                if single_mask.numel()
            )
        )
        if min_interp_features != 0:
            warnings.warn(
                "Minimum element in feature mask is not 0, shifting indices to"
                " start at 0.",
                stacklevel=2,
            )
            feature_mask_tuple = tuple(
                single_mask - min_interp_features for single_mask in feature_mask_tuple
            )

        #edited to consider background as a feature
        num_interp_features = _get_max_feature_index(feature_mask_tuple)
        if attribute_background:
            num_interp_features += 1
            
    return feature_mask_tuple, num_interp_features




In [1518]:
class LimeWithCustomArgumentToForwardFunc(LimeBaseWithCustomArgumentToForwardFunc):
    r"""
    Here we create a modification of Lime class from Captum Library (https://captum.ai/api/_modules/captum/attr/_core/lime.html)
    This will just inherit our modified LimeBase class
    """

    def __init__(
        self,
        forward_func: Callable[..., Tensor],
        interpretable_model: Optional[Model] = None,
        # pyre-fixme[24]: Generic type `Callable` expects 2 type parameters.
        similarity_func: Optional[Callable] = None,
        # pyre-fixme[24]: Generic type `Callable` expects 2 type parameters.
        perturb_func: Optional[Callable] = None,
    ) -> None:
        r"""

        Args:


            forward_func (Callable): The forward function of the model or any
                    modification of it
            interpretable_model (Model, optional): Model object to train
                    interpretable model.

                    This argument is optional and defaults to SkLearnLasso(alpha=0.01),
                    which is a wrapper around the Lasso linear model in SkLearn.
                    This requires having sklearn version >= 0.23 available.

                    Other predefined interpretable linear models are provided in
                    captum._utils.models.linear_model.

                    Alternatively, a custom model object must provide a `fit` method to
                    train the model, given a dataloader, with batches containing
                    three tensors:

                    - interpretable_inputs: Tensor
                      [2D num_samples x num_interp_features],
                    - expected_outputs: Tensor [1D num_samples],
                    - weights: Tensor [1D num_samples]

                    The model object must also provide a `representation` method to
                    access the appropriate coefficients or representation of the
                    interpretable model after fitting.

                    Note that calling fit multiple times should retrain the
                    interpretable model, each attribution call reuses
                    the same given interpretable model object.
            similarity_func (Callable, optional): Function which takes a single sample
                    along with its corresponding interpretable representation
                    and returns the weight of the interpretable sample for
                    training the interpretable model.
                    This is often referred to as a similarity kernel.

                    This argument is optional and defaults to a function which
                    applies an exponential kernel to the cosine distance between
                    the original input and perturbed input, with a kernel width
                    of 1.0.

                    A similarity function applying an exponential
                    kernel to cosine / euclidean distances can be constructed
                    using the provided get_exp_kernel_similarity_function in
                    captum.attr._core.lime.

                    Alternately, a custom callable can also be provided.
                    The expected signature of this callable is:

                    >>> def similarity_func(
                    >>>    original_input: Tensor or tuple[Tensor, ...],
                    >>>    perturbed_input: Tensor or tuple[Tensor, ...],
                    >>>    perturbed_interpretable_input:
                    >>>        Tensor [2D 1 x num_interp_features],
                    >>>    **kwargs: Any
                    >>> ) -> float or Tensor containing float scalar

                    perturbed_input and original_input will be the same type and
                    contain tensors of the same shape, with original_input
                    being the same as the input provided when calling attribute.

                    kwargs includes baselines, feature_mask, num_interp_features
                    (integer, determined from feature mask).
            perturb_func (Callable, optional): Function which returns a single
                    sampled input, which is a binary vector of length
                    num_interp_features, or a generator of such tensors.

                    This function is optional, the default function returns
                    a binary vector where each element is selected
                    independently and uniformly at LimeWithCustomArgumentToForwardFuncrandom. Custom
                    logic for selecting sampled binary vectors can
                    be implemented by providing a function with the
                    following expected signature:

                    >>> perturb_func(
                    >>>    original_input: Tensor or tuple[Tensor, ...],
                    >>>    **kwargs: Any
                    >>> ) -> Tensor [Binary 2D Tensor 1 x num_interp_features]
                    >>>  or generator yielding such tensors

                    kwargs includes baselines, feature_mask, num_interp_features
                    (integer, determined from feature mask).

        """
        if interpretable_model is None:
            interpretable_model = SkLearnLasso(alpha=0.01)

        if similarity_func is None:
            similarity_func = get_exp_kernel_similarity_function()

        if perturb_func is None:
            perturb_func = default_perturb_func

        LimeBaseWithCustomArgumentToForwardFunc.__init__(
            self,
            forward_func,
            interpretable_model,
            similarity_func,
            perturb_func,
            True,
            default_from_interp_rep_transform,
            None,
        )

    @log_usage(part_of_slo=True)
    def attribute(  # type: ignore
        self,
        inputs: TensorOrTupleOfTensorsGeneric,
        baselines: BaselineType = None,
        target: TargetType = None,
        additional_forward_args: Optional[object] = None,
        feature_mask: Union[None, Tensor, Tuple[Tensor, ...]] = None,
        n_samples: int = 25,
        perturbations_per_eval: int = 1,
        return_input_shape: bool = True,
        show_progress: bool = False,
    ) -> TensorOrTupleOfTensorsGeneric:
        r"""
        This method attributes the output of the model with given target index
        (in case it is provided, otherwise it assumes that output is a
        scalar) to the inputs of the model using the approach described above,
        training an interpretable model and returning a representation of the
        interpretable model.

        It is recommended to only provide a single example as input (tensors
        with first dimension or batch size = 1). This is because LIME is generally
        used for sample-based interpretability, training a separate interpretable
        model to explain a model's prediction on each individual example.

        A batch of inputs can also be provided as inputs, similar to
        other perturbation-based attribution methods. In this case, if forward_fn
        returns a scalar per example, attributions will be computed for each
        example independently, with a separate interpretable model trained for each
        example. Note that provided similarity and pertforward_funcurbation functions will be
        provided each example separately (first dimension = 1) in this case.
        If forward_fn returns a scalar per batch (e.g. loss), attributions will
        still be computed using a single interpretable model for the full batch.
        In this case, similarity and perturbation functions will be provided the
        same original input containing the full batch.

        The number of interpretable features is determined from the provided
        feature mask, or if none is provided, from the default feature mask,
        which considers each scalar input as a separate feature. It is
        generally recommended to provide a feature mask which groups features
        into a small number of interpretable features / components (e.g.
        superpixels in images).

        Args:

            inputs (Tensor or tuple[Tensor, ...]): Input for which LIME
                        is computed. If forward_func takes a single
                        tensor as input, a single input tensor should be provided.
                        If forward_func takes multiple tensors as input, a tuple
                        of the input tensors should be provided. It is assumed
                        that for all given input tensors, dimension 0 corresponds
                        to the number of examples, and if multiple input tensors
                        are provided, the examples must be aligned appropriately.
            baselines (scalar, Tensor, tuple of scalar, or Tensor, optional):
                        Baselines define reference value which replaces each
                        feature when the corresponding interpretable feature
                        is set to 0.
                        Baselines can be provided as:

                        - a single tensor, if inputs is a single tensor, with
                          exactly the same dimensions as inputs or the first
                          dimension is one and the remaining dimensions match
                          with inputs.

                        - a single scalar, if inputs is a single tensor, which will
                          be broadcasted for each input value in input tensor.

                        - a tuple of tensors or scalars, the baseline corresponding
                          to each tensor in the inputs' tuple can be:

                          - either a tensor with matching dimensions to
                            corresponding tensor in the inputs' tuple
                            or the first dimension is one and the remaining
                            dimensions match with the corresponding
                            input tensor.

                          - or a scalar, corresponding to a tensor in the
                            inputs' tuple. This scalar value is broadcasted
                            for corresponding input tensor.

                        In the cases when `baselines` iforward_funcs not provided, we internally
                        use zero scalar corresponding to each input tensor.
                        Default: None
            target (int, tuple, Tensor, or list, optional): Output indices for
                        which surrogate model is trained
                        (for classification cases,
                        this is usually the target class).
                        If the network returns a scalar value per example,
                        no target index is necessary.
                        For general 2D outputs, targets can be either:

                        - a single integer or a tensor containing a single
                          integer, which is applied to all input examples

                        - a list of integers or a 1D tensor, with length matching
                          the number of examples in inputs (dim 0). Each integer
                          is applied as the target for the corresponding example.

                        For outputs with > 2 dimensions, targets can be either:

                        - A single tuple, which contains #output_dims - 1
                          elements. This target index is applied to all examples.

                        - A list of tuples with length equal to the number of
                          examples in inputs (dim 0), and each tuple containing
                          #output_dims - 1 elements. Each tuple is applied as the
                          target for the corresponding example.

                        Default: None
            additional_forward_args (Any, optional): If the forward function
                        requires additional arguments other than the inputs for
                        which attributions should not be computed, this argument
                        can be provided. It must be either a single additional
                        argument of a Tensor or arbitrary (non-tuple) type or a
                        tuple containing multiple additional arguments including
                        tensors or any arbitrary python types. These arguments
                        are provided to forward_func in order following the
                        arguments in inputs.
                        For a tensor, the first dimension of the tensor must
                        correspond to the number of examples. It will be
                        repeated for each of `n_steps` along the integrated
                        path. For all other types, the given argument is used
                        for all forward evaluations.
                        Note that attributions are not computed with respect
                        to these arguments.
                        Default: None
            feature_mask (Tensor or tuple[Tensor, ...], optional):
                        feature_mask defines a mask for the input, grouping
                        features which correspond to the same
                        interpretable feature. feature_mask
                        should contain the same number of tensors as inputs.
                        Each tensor should
                        be the same size as the corresponding input or
                        broadcastable to match the inpuforward_funct tensor. Values across
                        all tensors should be integers in the range 0 to
                        num_interp_features - 1, and indices corresponding to the
                        same feature should have the same value.
                        Note that features are grouped across tensors
                        (unlike feature ablation and occlusion), so
                        if the same index is used in different tensors, those
                        features are still grouped and added simultaneously.
                        If None, then a feature mask is constructed which assigns
                        each scalar within a tensor as a separate feature.
                        Default: None
            n_samples (int, optional): The number of samples of the original
                        model used to train the surrogate interpretable model.
                        Default: `50` if `n_samples` is not provided.
            perturbations_per_eval (int, optional): Allows multiple samples
                        to be processed simultaneously in one call to forward_fn.
                        Each forward pass will contain a maximum of
                        perturbations_per_eval * #examples samples.
                        For DataParallel models, each batch is split among the
                        available devices, so evaluations on each available
                        device contain at most
                        (perturbations_per_eval * #examples) / num_devices
                        samples.
                        If the forward function returns a single scalar per batch,
                        perturbations_per_eval must be set to 1.
                        Default: 1
            return_input_shape (bool, optional): Determines whether the returned
                        tensor(s) only contain the coefficients for each interp-
                        retable feature from the trained surrogate model, or
                        whether the returned attributions match the input shape.
                        When return_input_shape is True, the return type of attribute
                        matches the input shape, with each element containing the
                        coefficient of the corresponding interpretale feature.
                        All elements with the same value in the feature mask
                        will contain the same coefficient in the returned
                        attributions.
                        If forward_func returns a single element per batch, then the
                        first dimension of each tensor will be 1, and the remaining
                        dimensions will have the same shape as the original input
                        tensor.
                        If return_input_shape is False, a 1D
                        tensor is returned, containing only the coefficients
                        of the trained interpreatable models, with length
                        num_interp_features.
            show_progress (bool, optional): Displays the progress of computation.
                        It will try to use tqdm if available for advanced features
                        (e.g. time estimation). Otherwise, it will fallback to
                        a simple output of progress.
                        Default: False

        Returns:
            *Tensor* or *tuple[Tensor, ...]* of **attributions**:
            - **attributions** (*Tensor* or *tuple[Tensor, ...]*):
                        The attributions with respect to each input feature.
                        If return_input_shape = True, attributions will be
                        the same size as the provided inputs, with each value
                        providing the coefficient of the corresponding
                        interpretale feature.
                        If return_input_shape is False, a 1D
                        tensor is returned, containing only the coefficients
                        of the trained interpreatable models, with length
                        num_interp_features.
        Examples::

            >>> # SimpleClassifier takes a single input tensor of size Nx4x4,
            >>> # and returns an Nx3 tensor of class probabilities.
            >>> net = SimpleClassifier()

            >>> # Generating random input with size 1 x 4 x 4
            >>> input = torch.randn(1, 4, 4)

            >>> # Defining Lime interpreter
            >>> lime = Lime(net)
            >>> # Computes attribution, with each of the 4 x 4 = 16
            >>> # features as a separate interpretable feature
            >>> attr = lime.attribute(input, target=1, n_samples=200)

            >>> # Alternatively, we can group each 2x2 square of the inputs
            >>> # as one 'interpretable' feature and perturb them together.
            >>> # This can be done by creating a feature mask as follows, which
            >>> # defines the feature groups, e.g.:
            >>> # +---+---+---+---+
            >>> # | 0 | 0 | 1 | 1 |
            >>> # +---+---+---+---+
            >>> # | 0 | 0 | 1 | 1 |
            >>> # +---+---+---+---+
            >>> # | 2 | 2 | 3 | 3 |
            >>> # +---+---+---+---+
            >>> # | 2 | 2 | 3 | 3 |
            >>> # +---+---+---+---+
            >>> # With this mask, all inputs with the same value are set to their
            >>> # baseline value, when the corresponding binary interpretable
            >>> # feature is set to 0.
            >>> # The attributions can be calculated as follows:
            >>> # feature mask has dimensions 1 x 4 x 4
            >>> feature_mask = torch.tensor([[[0,0,1,1],[0,0,1,1],
            >>>                             [2,2,3,3],[2,2,3,3]]])

            >>> # Computes interpretable model and returning attributions
            >>> # matching input shape.
            >>> attr = lime.attribute(input, target=1, feature_mask=feature_mask)
        """
        return self._attribute_kwargs(
            inputs=inputs,
            baselines=baselines,
            target=target,
            additional_forward_args=additional_forward_args,
            feature_mask=feature_mask,
            n_samples=n_samples,
            perturbations_per_eval=perturbations_per_eval,
            return_input_shape=return_input_shape,
            show_progress=show_progress,
        )

    # pyre-fixme[24] Generic type `Callable` expects 2 type parameters.
    def attribute_future(self) -> Callable:
        return super().attribute_future()

    def _attribute_kwargs(  # type: ignore
        self,
        inputs: TensorOrTupleOfTensorsGeneric,
        baselines: BaselineType = None,
        target: TargetType = None,
        additional_forward_args: Optional[object] = None,
        feature_mask: Union[None, Tensor, Tuple[Tensor, ...]] = None,
        attribute_background: bool = True,
        n_samples: int = 25,
        perturbations_per_eval: int = 1,
        return_input_shape: bool = True,
        monitor_log_path: str | None = None,
        monitor_convergence_step: int | None = 20,
        monitor_local_accuracy_step: int | None = 50,
        show_progress: bool = False,
        **kwargs: object,
    ) -> TensorOrTupleOfTensorsGeneric:
        is_inputs_tuple = _is_tuple(inputs)
        formatted_inputs, baselines = _format_input_baseline(inputs, baselines)
        bsz = formatted_inputs[0].shape[0]

        feature_mask, num_interp_features = construct_feature_mask(
            feature_mask, formatted_inputs
        )

        if num_interp_features > 10000:
            warnings.warn(
                "Attempting to construct interpretable model with > 10000 features."
                "This can be very slow or lead to OOM issues. Please provide a feature"
                "mask which groups input features to reduce the number of interpretable"
                "features. ",
                stacklevel=1,
            )

        coefs: Tensor
        if bsz > 1:
            test_output = _run_forward(
                self.forward_func, inputs, target, additional_forward_args
            )
            if isinstance(test_output, Tensor) and torch.numel(test_output) > 1:
                if torch.numel(test_output) == bsz:
                    warnings.warn(
                        "You are providing multiple inputs for Lime / Kernel SHAP "
                        "attributions. This trains a separate interpretable model "
                        "for each example, which can be time consuming. It is "
                        "recommended to compute attributions for one example at a "
                        "time.",
                        stacklevel=1,
                    )
                    output_list = []
                    for (
                        curr_inps,
                        curr_target,
                        curr_additional_args,
                        curr_baselines,
                        curr_feature_mask,
                    ) in _batch_example_iterator(
                        bsz,
                        formatted_inputs,
                        target,
                        additional_forward_args,# -----> CAN BE ALSO BATCHED AUTOMATICALLY BY THE LIBRARY ITERATOR
                        baselines,
                        feature_mask,
                    ):
                        coefs = super().attribute.__wrapped__(
                            self,
                            inputs=curr_inps if is_inputs_tuple else curr_inps[0],
                            target=curr_target,
                            additional_forward_args=curr_additional_args,
                            n_samples=n_samples,
                            perturbations_per_eval=perturbations_per_eval,
                            baselines=(
                                curr_baselines if is_inputs_tuple else curr_baselines[0]
                            ),
                            feature_mask=(
                                curr_feature_mask
                                if is_inputs_tuple
                                else curr_feature_mask[0]
                            ),
                            num_interp_features=num_interp_features,
                            show_progress=show_progress,
                            **kwargs,
                        )
                        if return_input_shape:
                            output_list.append(
                                self._convert_output_shape(
                                    curr_inps,
                                    curr_feature_mask,
                                    coefs,
                                    num_interp_features,
                                    attribute_background,
                                    is_inputs_tuple,
                                )
                            )
                        else:
                            output_list.append(coefs.reshape(1, -1))  # type: ignore

                    return _reduce_list(output_list)
                else:
                    raise AssertionError(
                        "Invalid number of outputs, forward function should return a"
                        "scalar per example or a scalar per input batch."
                    )
            else:
                assert perturbations_per_eval == 1, (
                    "Perturbations per eval must be 1 when forward function"
                    "returns single value per batch!"
                )

        coefs = super().attribute.__wrapped__(
            self,
            inputs=inputs,
            target=target,
            additional_forward_args=additional_forward_args,
            n_samples=n_samples,
            perturbations_per_eval=perturbations_per_eval,
            baselines=baselines if is_inputs_tuple else baselines[0],
            feature_mask=feature_mask if is_inputs_tuple else feature_mask[0],
            num_interp_features=num_interp_features,
            attribute_background=attribute_background,
            monitor_log_path = monitor_log_path,
            monitor_convergence_step = monitor_convergence_step,
            monitor_local_accuracy_step = monitor_local_accuracy_step,
            show_progress=show_progress,
            **kwargs,
        )
        if return_input_shape:
            # pyre-fixme[7]: Expected `TensorOrTupleOfTensorsGeneric` but got
            #  `Tuple[Tensor, ...]`.
            return self._convert_output_shape(
                formatted_inputs,
                feature_mask,
                coefs,
                num_interp_features,
                attribute_background,
                is_inputs_tuple,
    
            leading_dim_one=(bsz > 1),
            )
        else:
            return coefs

    @typing.overload
    def _convert_output_shape(
        self,
        formatted_inp: Tuple[Tensor, ...],
        feature_mask: Tuple[Tensor, ...],
        coefs: Tensor,
        num_interp_features: int,
        is_inputs_tuple: Literal[True],
        leading_dim_one: bool = False,
    ) -> Tuple[Tensor, ...]: ...

    @typing.overload
    def _convert_output_shape(  # type: ignore
        self,
        formatted_inp: Tuple[Tensor, ...],
        feature_mask: Tuple[Tensor, ...],
        coefs: Tensor,
        num_interp_features: int,
        is_inputs_tuple: Literal[False],
        leading_dim_one: bool = False,
    ) -> Tensor: ...

    @typing.overload
    def _convert_output_shape(
        self,
        formatted_inp: Tuple[Tensor, ...],
        feature_mask: Tuple[Tensor, ...],
        coefs: Tensor,
        num_interp_features: int,
        is_inputs_tuple: bool,
        leading_dim_one: bool = False,
    ) -> Union[Tensor, Tuple[Tensor, ...]]: ...

    def _convert_output_shape(
        self,
        formatted_inp: Tuple[Tensor, ...],
        feature_mask: Tuple[Tensor, ...],
        coefs: Tensor,
        num_interp_features: int,
        attribute_background: bool,
        is_inputs_tuple: bool,
        leading_dim_one: bool = False,
    ) -> Union[Tensor, Tuple[Tensor, ...]]:
        coefs = coefs.flatten()
        attr = [
            torch.zeros_like(single_inp, dtype=torch.float)
            for single_inp in formatted_inp
        ]
        for tensor_ind in range(len(formatted_inp)):
            # modified (background)
            interp_space_labels = range(num_interp_features)
            if attribute_background:
                input_space_feature_labels = range(num_interp_features)
            else:
                input_space_feature_labels = range(1, num_interp_features+1)
            for interp_label, input_label in zip(interp_space_labels, input_space_feature_labels):
                attr[tensor_ind] += (
                    coefs[interp_label].item()
                    * (feature_mask[tensor_ind] == input_label).float()
                )
        if leading_dim_one:
            for i in range(len(attr)):
                attr[i] = attr[i][0:1]
        return _format_output(is_inputs_tuple, tuple(attr))


In [1519]:
#!/usr/bin/env python3

# pyre-strict

from typing import Callable, cast, Generator, Optional, Tuple, Union

import torch
from captum._utils.models.linear_model import SkLearnLinearRegression
from captum._utils.typing import BaselineType, TargetType, TensorOrTupleOfTensorsGeneric
from captum.attr._core.lime import Lime #,construct_feature_mask modified to manage background
from captum.attr._utils.common import _format_input_baseline
from captum.log import log_usage
from torch import Tensor
from torch.distributions.categorical import Categorical


class KernelShapWithMask(LimeWithCustomArgumentToForwardFunc):
    r"""
    Kernel SHAP is a method that uses the LIME framework to compute
    Shapley Values. Setting the loss function, weighting kernel and
    regularization terms appropriately in the LIME framework allows
    theoretically obtaining Shapley Values more efficiently than
    directly computing Shapley Values.

    More information regarding this method and proof of equivalence
    can be found in the original paper here:
    https://arxiv.org/abs/1705.07874
    """

    def __init__(self, 
                 forward_func: Callable[..., Tensor],
                 surrogate_model: str = "linear regression",
                 alpha_surrogate: float = 0.01,
                 max_iter_surrogate: int = 1000
                ) -> None:
        r"""
        Args:

            forward_func (Callable): The forward function of the model or
                        any modification of it.
        """
        if surrogate_model == "linear regression":
            interpretable_model = SkLearnLinearRegression()
        elif surrogate_model == "lasso": 
            interpretable_model = SkLearnLasso(alpha=alpha_surrogate, max_iter=max_iter_surrogate)
            
        LimeWithCustomArgumentToForwardFunc.__init__(
            self,
            forward_func,
            interpretable_model=interpretable_model,
            similarity_func=self.kernel_shap_similarity_kernel,
            perturb_func=self.kernel_shap_perturb_generator,
        )
        self.inf_weight = 1000000.0

    @log_usage(part_of_slo=True)
    def attribute(  # type: ignore
        self,
        inputs: TensorOrTupleOfTensorsGeneric,
        baselines: BaselineType = None,
        target: TargetType = None,
        additional_forward_args: Optional[object] = None,
        feature_mask: Union[None, Tensor, Tuple[Tensor, ...]] = None,
        attribute_background: bool = True,
        n_samples: int = 25,
        perturbations_per_eval: int = 1,
        return_input_shape: bool = True,
        monitor_log_path: str | None = None,
        monitor_convergence_step: int | None = 20,
        monitor_local_accuracy_step: int | None = 50,
        show_progress: bool = False,
    ) -> TensorOrTupleOfTensorsGeneric:
        r"""
        This method attributes the output of the model with given target index
        (in case it is provided, otherwise it assumes that output is a
        scalar) to the inputs of the model using the approach described above,
        training an interpretable model based on KernelSHAP and returning a
        representation of the interpretable model.

        It is recommended to only provide a single example as input (tensors
        with first dimension or batch size = 1). This is because LIME / KernelShap
        is generally used for sample-based interpretability, training a separate
        interpretable model to explain a model's prediction on each individual example.

        A batch of inputs can also be provided as inputs, similar to
        other perturbation-based attribution methods. In this case, if forward_fn
        returns a scalar per example, attributions will be computed for each
        example independently, with a separate interpretable model trained for each
        example. Note that provided similarity and perturbation functions will be
        provided each example separately (first dimension = 1) in this case.
        If forward_fn returns a scalar per batch (e.g. loss), attributions will
        still be computed using a single interpretable model for the full batch.
        In this case, similarity and perturbation functions will be provided the
        same original input containing the full batch.

        The number of interpretable features is determined from the provided
        feature mask, or if none is provided, from the default feature mask,
        which considers each scalar input as a separate feature. It is
        generally recommended to provide a feature mask which groups features
        into a small number of interpretable features / components (e.g.
        superpixels in images).

        Args:

            inputs (Tensor or tuple[Tensor, ...]): Input for which KernelShap
                        is computed. If forward_func takes a single
                        tensor as input, a single input tensor should be provided.
                        If forward_func takes multiple tensors as input, a tuple
                        of the input tensors should be provided. It is assumed
                        that for all given input tensors, dimension 0 corresponds
                        to the number of examples, and if multiple input tensors
                        are provided, the examples must be aligned appropriately.
            baselines (scalar, Tensor, tuple of scalar, or Tensor, optional):
                        Baselines define the reference value which replaces eachconv_dist_L1 and delta_shap
                        feature when the corresponding interpretable feature
                        is set to 0.
                        Baselines can be provided as:

                        - a single tensor, if inputs is a single tensor, with
                          exactly the same dimensions as inputs or the first
                          dimension is one and the remaining dimensions match
                          with inputs.

                        - a single scalar, if inputs is a single tensor, which will
                          be broadcasted for each input value in input tensor.

                        - a tuple of tensors or scalars, the baseline corresponding
                          to each tensor in the inputs' tuple can be:

                          - either a tensor with matching dimensions to
                            corresponding tensor in the inputs' tuple
                            or the first dimension is one and the remaining
                            dimensions match with the corresponding
                            input tensor.

                          - or a scalar, corresponding to a tensor in the
                            inputs' tuple. This scalar value is broadcasted
                            for corresponding input tensor.

                        In the cases when `baselines` is not provided, we internally
                        use zero scalar corresponding to each input tensor.
                        Default: None
            target (int, tuple, Tensor, or list, optional): Output indices for
                        which surrogate model is trained
                        (for classification cases,
                        this is usually the target class).
                        If the network returns a scalar value per example,
                        no target index is necessary.
                        For general 2D outputs, targets can be either:

                        - a single integer or a tensor containing a single
                          integer, which is applied to all input examples

                        - a list of integers or a 1D tensor, with length matching
                          the number of examples in inputs (dim 0). Each integer
                          is applied as the target for the corresponding example.

                        For outputs with > 2 dimensions, targets can be either:

                        - A single tuple, which contains #output_dims - 1
                          elements. This target index is applied to all examples.

                        - A list of tuples with length equal to the number of
                          examples in inputs (dim 0), and each tuple containing
                          #output_dims - 1 elements. Each tuple is applied as the
                          target for the corresponding example.

                        Default: None
            additional_forward_args (Any, optional): If the forward function
                        requires additional arguments other than the inputs for
                        which attributions should not be computed, this argument
                        can be provided. It must be either a single additional
                        argument of a Tensor or arbitrary (non-tuple) type or a
                        tuple containing multiple additional arguments including
                        tensors or any arbitrary python types. These arguments
                        are provided to forward_func in order following the
                        arguments in inputs.
                        For a tensor, the first dimension of the tensor must
                        correspond to the number of examples. It will be
                        repeated for each of `n_steps` along the integrated
                        path. For all other types, the given argument is used
                        for all forward evaluations.
                        Note that attributions are not computed with respect
                        to these arguments.
                        Default: None
            feature_mask (Tensor or tuple[Tensor, ...], optional):
                        feature_mask defines a mask for the input, grouping
                        features which correspond to the same
                        interpretable feature. feature_mask
                        should contain the same number of tensors as inputs.
                        Each tensor should
                        be the same size as the corresponding input or
                        broadcastable to match the input tensor. Values across
                        all tensors should be integers in the range 0 to
                        num_interp_features - 1, and indices corresponding to the
                        same feature should have the same value.
                        If attribute_background is set to False, the background feature
                        to ignore is assumed to be the first (0 value).
                        Note that features are grouped across tensors
                        (unlike feature ablation and occlusion), so
                        if the same index is used in different tensors, those
                        features are still grouped and added simultaneously.
                        If None, then a feature mask is constructed which assigns
                        each scalar within a tensor as a separate feature.
                        Default: None
            attribute_background (bool, optional): Determines wether the feature
                        corresponding to 0, i.e. the background, must be considered as a
                        regular feature to be attributed, or must be ignored.
                        Default: True
            n_samples (int, optional): The number of samples of the original
                        model used to train the surrogate interpretable model.
                        Default: `50` if `n_samples` is not provided.
            perturbations_per_eval (int, optional): Allows multiple samples
                        to be processed simultaneously in one call to forward_fn.
                        Each forward pass will contain a maximum of
                        perturbations_per_eval * #examples samples.
                        For DataParallel models, each batch is split among the
                        available devices, so evaluations on each available
                        device contain at most
                        (perturbations_per_eval * #examples) / num_devices
                        samples.
                        If the forward function returns a single scalar per batch,
                        perturbations_per_eval must be set to 1.
                        Default: 1
            return_input_shape (bool, optional): Determines whether the returned
                        tensor(s) only contain the coefficients for each interp-
                        retable feature from the trained surrogate model, or
                        whether the returned attributions match the input shape.
                        When return_input_shape is True, the return type of attribute
                        matches the input shape, with each element containing the
                        coefficient of the corresponding interpretable feature.
                        All elements with the same value in the feature mask
                        will contain the same coefficient in the returned
                        attributions. In case of attribute_background set to False, 
                        the background element (which won't have a corresponding
                        element in the interpretable feature vector) will be set to 0.
                        If return_input_shape is False, a 1D
                        tensor is returned, containing only the coefficients
                        of the trained interpretable model, with length
                        num_interp_features.
            monitor_log_path (str, optional): Path to the log file for monitoring convergence.
                        if None, no monitoring is performed.
                        Default: None
            monitor_convergence_step (int, optional): Number of iterations over which
                        the difference among two attribution is computerd.
                        Default: 20
            monitor_local_accuracy_step (int, optional): Number of iterations over which
                        the local accuracy of an attribution is computerd.
                        Default: 50
            show_progress (bool, optional): Displays the progress of computation.
                        It will try to use tqdm if available for advanced features
                        (e.g. time estimation). Otherwise, it will fallback to
                        a simple output of progress.
                        Default: False

        Returns:
            *Tensor* or *tuple[Tensor, ...]* of **attributions**:
            - **attributions** (*Tensor* or *tuple[Tensor, ...]*):
                        The attributions with respect to each input feature.
                        If return_input_shape = True, attributions will be
                        the same size as the provided inputs, with each value
                        providing the coefficient of the corresponding
                        interpretale feature.
                        If return_input_shape is False, a 1D
                        tensor is returned, containing only the coefficients
                        of the trained interpreatable models, with length
                        num_interp_features.
        Examples::
            >>> # SimpleClassifier takes a single input tensor of size Nx4x4,
            >>> # and returns an Nx3 tensor of class probabilities.
            >>> net = SimpleClassifier()

            >>> # Generating random input with size 1 x 4 x 4
            >>> input = torch.randn(1, 4, 4)

            >>> # Defining KernelShap interpreter
            >>> ks = KernelShap(net)
            >>> # Computes attribution, with each of the 4 x 4 = 16
            >>> # features as a separate interpretable feature
            >>> attr = ks.attribute(input, target=1, n_samples=200)

            >>> # Alternatively, we can group each 2x2 square of the inputs
            >>> # as one 'interpretable' feature and perturb them together.
            >>> # This can be done by creating a feature mask as follows, which
            >>> # defines the feature groups, e.g.:
            >>> # +---+---+---+---+
            >>> # | 0 | 0 | 1 | 1 |
            >>> # +---+---+---+---+
            >>> # | 0 | 0 | 1 | 1 |
            >>> # +---+---+---+---+
            >>> # | 2 | 2 | 3 | 3 |
            >>> # +---+---+---+---+
            >>> # | 2 | 2 | 3 | 3 |
            >>> # +---+---+---+---+
            >>> # With this mask, all inputs with the same value are set to their
            >>> # baseline value, when the corresponding binary interpretable
            >>> # feature is set to 0.
            >>> # The attributions can be calculated as follows:
            >>> # feature mask has dimensions 1 x 4 x 4
            >>> feature_mask = torch.tensor([[[0,0,1,1],[0,0,1,1],
            >>>                             [2,2,3,3],[2,2,3,3]]])

            >>> # Computes KernelSHAP attributions with feature mask.
            >>> attr = ks.attribute(input, target=1, feature_mask=feature_mask)
        """
        formatted_inputs, baselines = _format_input_baseline(inputs, baselines)
        feature_mask, num_interp_features = construct_feature_mask(
            feature_mask, formatted_inputs, attribute_background
        )
        num_features_list = torch.arange(num_interp_features, dtype=torch.float)
        denom = num_features_list * (num_interp_features - num_features_list)
        probs = torch.tensor((num_interp_features - 1)) / denom
        probs[0] = 0.0
        return self._attribute_kwargs(
            inputs=inputs,
            baselines=baselines,
            target=target,
            additional_forward_args=additional_forward_args,
            feature_mask=feature_mask,
            attribute_background=attribute_background,
            n_samples=n_samples,
            perturbations_per_eval=perturbations_per_eval,
            return_input_shape=return_input_shape,
            num_select_distribution=Categorical(probs),
            monitor_log_path=monitor_log_path,
            monitor_convergence_step=monitor_convergence_step,
            monitor_local_accuracy_step=monitor_local_accuracy_step,
            show_progress=show_progress,
        )

    # pyre-fixme[24] Generic type `Callable` expects 2 type parameters.
    def attribute_future(self) -> Callable:
        r"""
        This method is not implemented for KernelShap.
        """
        raise NotImplementedError("attribute_future is not implemented for KernelShap")

    def kernel_shap_similarity_kernel(
        self,
        _,
        __,
        interpretable_sample: Tensor,
        **kwargs: object,
    ) -> Tensor:
        assert (
            "num_interp_features" in kwargs
        ), "Must provide num_interp_features to use default similarity kernel"
        num_selected_features = int(interpretable_sample.sum(dim=1).item())
        num_features = kwargs["num_interp_features"]
        if num_selected_features == 0 or num_selected_features == num_features:
            # weight should be theoretically infinite when
            # num_selected_features = 0 or num_features
            # enforcing that trained linear model must satisfy
            # end-point criteria. In practice, it is sufficient to
            # make this weight substantially larger so setting this
            # weight to 1000000 (all other weights are 1).
            similarities = self.inf_weight
        else:
            similarities = 1.0
        return torch.tensor([similarities])

    def kernel_shap_perturb_generator(
        self,
        original_inp: Union[Tensor, Tuple[Tensor, ...]],
        **kwargs: object,
    ) -> Generator[Tensor, None, None]:
        r"""
        Perturbations are sampled by the following process:
         - Choose k (number of selected features), based on the distribution
                p(k) = (M - 1) / (k * (M - k))

            where M is the total number of features in the interpretable space

         - Randomly select a binary vector with k ones, each sample is equally
            likely. This is done by generating a random vector of normal
            values and thresholding based on the top k elements.

         Since there are M choose k vectors with k ones, this weighted sampling
         is equivalent to applying the Shapley kernel for the sample weight,
         defined as:
         k(M, k) = (M - 1) / (k * (M - k) * (M choose k))
        """
        assert (
            "num_select_distribution" in kwargs and "num_interp_features" in kwargs
        ), (
            "num_select_distribution and num_interp_features are necessary"
            " to use kernel_shap_perturb_func"
        )
        if isinstance(original_inp, Tensor):
            device = original_inp.device
        else:
            device = original_inp[0].device
        num_features = cast(int, kwargs["num_interp_features"])
        yield torch.ones(1, num_features, device=device, dtype=torch.long)
        yield torch.zeros(1, num_features, device=device, dtype=torch.long)
        while True:
            num_selected_features = cast(
                Categorical, kwargs["num_select_distribution"]
            ).sample()
            rand_vals = torch.randn(1, num_features)
            threshold = torch.kthvalue(
                rand_vals, num_features - num_selected_features
            ).values.item()
            yield (rand_vals > threshold).to(device=device).long()

   

# Next step: define regular, fixed size superpixels and try to compute the attributions of each of them
So we also need to define a metric to compare, since segmentation explanations, differently from classification, is intrinsically ambiguous. For example, let's select a priori a single region of the segmentation output, and use the average of these pixels to compute the impact of perturbations.

### 4. Face-centered cubic (FCC) lattice induced supervoxel assignment
-> more *isotropic* than simple cubes

### 4.1 affine transformation to translate isotropy from voxel space into the original geometrical space (measured in mm)

### 4.2 Try to apply the original algorithm FCC it to an affine transformed volume that has the same proportion as the .nii in the physical space. Transform->apply the algorithm to derive the map-> back transform the map onto the voxel space



In [1520]:
import numpy as np
import nibabel as nib
from scipy.spatial import cKDTree

def generate_FCC_supervoxel_map(img, S=200.0):
    """
    Generate a supervoxel map using FCC tessellation in physical space (original version).
    
    Args:
        img: Nibabel NIfTI image object
        S (float): Desired supervoxel size in millimeters (default: 200.0)
    
    Returns:
        supervoxel_map: 3D NumPy array with integer labels for supervoxels
    """
    # Load volume and affine from image
    volume = img.get_fdata()
    affine = img.affine
    W, H, D = volume.shape

    # Compute the physical bounding box of the volume
    corners_voxel = np.array([
        [0, 0, 0],
        [W-1, 0, 0],
        [0, H-1, 0],
        [0, 0, D-1],
        [W-1, H-1, 0],
        [W-1, 0, D-1],
        [0, H-1, D-1],
        [W-1, H-1, D-1]
    ])
    corners_hom = np.hstack((corners_voxel, np.ones((8, 1))))
    corners_physical = (affine @ corners_hom.T).T[:, :3]
    min_xyz = corners_physical.min(axis=0)
    max_xyz = corners_physical.max(axis=0)

    # Generate FCC lattice centers in physical space
    a = S * np.sqrt(2)
    factor = 2 / a
    p_min = int(np.floor(factor * min_xyz[0])) - 1
    p_max = int(np.ceil(factor * max_xyz[0])) + 1
    q_min = int(np.floor(factor * min_xyz[1])) - 1
    q_max = int(np.ceil(factor * max_xyz[1])) + 1
    r_min = int(np.floor(factor * min_xyz[2])) - 1
    r_max = int(np.ceil(factor * max_xyz[2])) + 1

    # Create grid of possible indices
    p_vals = np.arange(p_min, p_max + 1)
    q_vals = np.arange(q_min, q_max + 1)
    r_vals = np.arange(r_min, r_max + 1)
    P, Q, R = np.meshgrid(p_vals, q_vals, r_vals, indexing='ij')
    P = P.flatten()
    Q = Q.flatten()
    R = R.flatten()

    # Filter for FCC lattice points (sum of indices is even)
    mask = (P + Q + R) % 2 == 0
    P = P[mask]
    Q = Q[mask]
    R = R[mask]

    # Compute physical coordinates of centers
    centers = np.column_stack((P * a / 2, Q * a / 2, R * a / 2))

    # Keep only centers within the bounding box
    inside = ((centers[:, 0] >= min_xyz[0]) & (centers[:, 0] <= max_xyz[0]) &
              (centers[:, 1] >= min_xyz[1]) & (centers[:, 1] <= max_xyz[1]) &
              (centers[:, 2] >= min_xyz[2]) & (centers[:, 2] <= max_xyz[2]))
    centers = centers[inside]

    # Check if any centers were generated
    print(f"Number of supervoxel centers: {len(centers)}")
    if len(centers) == 0:
        raise ValueError("No supervoxel centers generated. Try reducing S.")

    # Generate voxel indices and transform to physical coordinates
    voxel_indices = np.indices((W, H, D)).reshape(3, -1).T  # shape (W*H*D, 3)
    voxel_indices_hom = np.hstack((voxel_indices, np.ones((voxel_indices.shape[0], 1))))  # shape (W*H*D, 4)
    physical_coords = (affine @ voxel_indices_hom.T).T[:, :3]  # shape (W*H*D, 3)

    # Assign each voxel to the nearest supervoxel center
    tree = cKDTree(centers)
    _, labels = tree.query(physical_coords)

    # Create the supervoxel map
    supervoxel_map = labels.reshape((W, H, D)).astype(np.int32)

    return supervoxel_map

## Combine FCC regularity with organ context: Organ-aware FCC supervoxels

In [1521]:
import numpy as np
import nibabel as nib
from scipy.spatial import cKDTree
from collections import defaultdict

def generate_FCC_organs_supervoxel_map(volume_img, organ_img, S, compute_statistics=False):
    """
    Generate organ-aware supervoxel map using FCC tessellation.
    
    Args:
        volume_img: Nibabel NIfTI image object (intensity volume)
        organ_img: Nibabel NIfTI image object (organ labels, 0=background)
        S: Desired supervoxel size in millimeters
    
    Returns:
        tuple: (supervoxel_map, organ_table, statistics)
            - supervoxel_map: 3D array with supervoxel IDs (0=background)
            - organ_table: dict {supervoxel_id: organ_label}
            - statistics: dict with per-supervoxel statistics
    """
    # Load data
    volume = volume_img.get_fdata()
    organs = organ_img.get_fdata().astype(np.int32)
    affine = volume_img.affine
    W, H, D = volume.shape
    
    # Verify dimensions match
    if volume.shape != organs.shape:
        raise ValueError("Volume and organ images must have the same dimensions")
    
    # Compute physical bounding box
    corners_voxel = np.array([
        [0, 0, 0], [W-1, 0, 0], [0, H-1, 0], [0, 0, D-1],
        [W-1, H-1, 0], [W-1, 0, D-1], [0, H-1, D-1], [W-1, H-1, D-1]
    ])
    corners_hom = np.hstack((corners_voxel, np.ones((8, 1))))
    corners_physical = (affine @ corners_hom.T).T[:, :3]
    min_xyz = corners_physical.min(axis=0)
    max_xyz = corners_physical.max(axis=0)

    # Generate FCC lattice centers in physical space
    a = S * np.sqrt(2)
    factor = 2 / a
    p_min = int(np.floor(factor * min_xyz[0])) - 1
    p_max = int(np.ceil(factor * max_xyz[0])) + 1
    q_min = int(np.floor(factor * min_xyz[1])) - 1
    q_max = int(np.ceil(factor * max_xyz[1])) + 1
    r_min = int(np.floor(factor * min_xyz[2])) - 1
    r_max = int(np.ceil(factor * max_xyz[2])) + 1

    # Create FCC lattice points efficiently
    centers = []
    for p in range(p_min, p_max + 1):
        for q in range(q_min, q_max + 1):
            for r in range(r_min, r_max + 1):
                if (p + q + r) % 2 == 0:  # FCC constraint
                    center = np.array([p * a / 2, q * a / 2, r * a / 2])
                    # Check if within bounding box
                    if (min_xyz[0] <= center[0] <= max_xyz[0] and
                        min_xyz[1] <= center[1] <= max_xyz[1] and
                        min_xyz[2] <= center[2] <= max_xyz[2]):
                        centers.append(center)
    
    centers = np.array(centers)
    print(f"Generated {len(centers)} FCC centers")
    
    if len(centers) == 0:
        raise ValueError("No FCC centers generated. Try reducing S.")
    
    # Process volume slice by slice to save memory
    supervoxel_map = np.zeros((W, H, D), dtype=np.int32)
    fcc_to_organs = defaultdict(set)  # Maps FCC center index to set of organ labels
    
    tree = cKDTree(centers)
    
    print("Assigning voxels to FCC centers...")
    for w in range(W):
        # Generate coordinates for this slice
        y_coords, z_coords = np.meshgrid(np.arange(H), np.arange(D), indexing='ij')
        slice_voxels = np.column_stack([
            np.full(H * D, w),
            y_coords.ravel(),
            z_coords.ravel()
        ])
        
        # Transform to physical space
        slice_physical = (affine @ np.column_stack([slice_voxels, np.ones(H * D)]).T).T[:, :3]
        
        # Assign to nearest FCC center
        _, fcc_labels = tree.query(slice_physical)
        fcc_labels = fcc_labels.reshape(H, D)
        
        # Store FCC assignments and collect organ labels per FCC center
        for h in range(H):
            for d in range(D):
                fcc_idx = fcc_labels[h, d]
                organ_label = organs[w, h, d]
                
                if organ_label != 0:  # Skip background
                    fcc_to_organs[fcc_idx].add(organ_label)
                
                # Temporarily store FCC index (will be converted to supervoxel ID later)
                supervoxel_map[w, h, d] = fcc_idx
    
    # Create supervoxel IDs: each (FCC_center, organ) pair gets unique ID
    print("Creating organ-aware supervoxels...")
    fcc_organ_to_supervoxel = {}
    supervoxel_to_organ = {}
    supervoxel_id = 1  # Start from 1 (0 reserved for background)
    
    for fcc_idx in sorted(fcc_to_organs.keys()):
        for organ_label in sorted(fcc_to_organs[fcc_idx]):
            fcc_organ_to_supervoxel[(fcc_idx, organ_label)] = supervoxel_id
            supervoxel_to_organ[supervoxel_id] = organ_label
            supervoxel_id += 1
    
    print(f"Created {len(supervoxel_to_organ)} organ-aware supervoxels")
    
    # Convert FCC assignments to final supervoxel IDs
    final_supervoxel_map = np.zeros((W, H, D), dtype=np.int32)
    
    for w in range(W):
        for h in range(H):
            for d in range(D):
                fcc_idx = supervoxel_map[w, h, d]
                organ_label = organs[w, h, d]
                
                if organ_label != 0:  # Non-background
                    key = (fcc_idx, organ_label)
                    if key in fcc_organ_to_supervoxel:
                        final_supervoxel_map[w, h, d] = fcc_organ_to_supervoxel[key]
                # Background voxels remain 0

    if compute_statistics:
        # Compute statistics
        print("Computing supervoxel statistics...")
        statistics = {}
        voxel_volume_mm3 = np.abs(np.linalg.det(affine[:3, :3]))
        
        for sv_id in supervoxel_to_organ.keys():
            mask = final_supervoxel_map == sv_id
            if np.any(mask):
                voxels = volume[mask]
                statistics[sv_id] = {
                    'organ_label': supervoxel_to_organ[sv_id],
                    'voxel_count': int(np.sum(mask)),
                    'volume_mm3': float(np.sum(mask) * voxel_volume_mm3),
                    'mean_intensity': float(np.mean(voxels)),
                    'std_intensity': float(np.std(voxels)),
                    'min_intensity': float(np.min(voxels)),
                    'max_intensity': float(np.max(voxels)),
                    'centroid_voxel': [float(x) for x in np.array(np.where(mask)).mean(axis=1)]
                }
        
        # Add background statistics
        background_mask = final_supervoxel_map == 0
        if np.any(background_mask):
            bg_voxels = volume[background_mask]
            statistics[0] = {
                'organ_label': 0,
                'voxel_count': int(np.sum(background_mask)),
                'volume_mm3': float(np.sum(background_mask) * voxel_volume_mm3),
                'mean_intensity': float(np.mean(bg_voxels)),
                'std_intensity': float(np.std(bg_voxels)),
                'min_intensity': float(np.min(bg_voxels)),
                'max_intensity': float(np.max(bg_voxels)),
                'centroid_voxel': [float(x) for x in np.array(np.where(background_mask)).mean(axis=1)]
            }
        
        # Print summary
        organ_counts = defaultdict(int)
        for sv_id, organ_label in supervoxel_to_organ.items():
            organ_counts[organ_label] += 1
        
        print("\nSupervoxel summary:")
        print(f"Background supervoxels: 1 (ID: 0)")
        for organ_label in sorted(organ_counts.keys()):
            print(f"Organ {organ_label}: {organ_counts[organ_label]} supervoxels")
    else:
        supervoxel_to_organ, statistics = None, None
    
    return final_supervoxel_map, supervoxel_to_organ, statistics


def save_organ_supervoxel_results(supervoxel_map, organ_table, statistics, 
                                volume_img, output_prefix="organ_supervoxels"):
    """
    Save supervoxel results to files.
    
    Args:
        supervoxel_map: 3D array with supervoxel IDs
        organ_table: dict mapping supervoxel_id -> organ_label  
        statistics: dict with supervoxel statistics
        volume_img: original volume image (for affine/header)
        output_prefix: prefix for output files
    """
    # Save supervoxel map as NIfTI
    supervoxel_img = nib.Nifti1Image(supervoxel_map.astype(np.int32), 
                                   volume_img.affine, volume_img.header)
    supervoxel_img.header.set_data_dtype(np.int32)
    nib.save(supervoxel_img, f"{output_prefix}_map.nii.gz")
    print(f"Saved supervoxel map: {output_prefix}_map.nii.gz")
    
    # Save organ table as CSV
    import csv
    with open(f"{output_prefix}_table.csv", 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['supervoxel_id', 'organ_label'])
        writer.writerow([0, 0])  # Background
        for sv_id in sorted(organ_table.keys()):
            writer.writerow([sv_id, organ_table[sv_id]])
    print(f"Saved organ table: {output_prefix}_table.csv")
    
    # Save statistics as CSV
    with open(f"{output_prefix}_stats.csv", 'w', newline='') as f:
        writer = csv.writer(f)
        header = ['supervoxel_id', 'organ_label', 'voxel_count', 'volume_mm3',
                 'mean_intensity', 'std_intensity', 'min_intensity', 'max_intensity',
                 'centroid_x', 'centroid_y', 'centroid_z']
        writer.writerow(header)
        
        for sv_id in sorted(statistics.keys()):
            stats = statistics[sv_id]
            row = [sv_id, stats['organ_label'], stats['voxel_count'], stats['volume_mm3'],
                  stats['mean_intensity'], stats['std_intensity'], 
                  stats['min_intensity'], stats['max_intensity']] + stats['centroid_voxel']
            writer.writerow(row)
    print(f"Saved statistics: {output_prefix}_stats.csv")

## Try SLIC for visual context aware supervoxels

### define a preprocessing routine to enhance SLIC results

In [1522]:
from skimage import exposure
from skimage.restoration import denoise_nl_means, estimate_sigma
from scipy.ndimage import gaussian_filter
from skimage.exposure import equalize_adapthist

def preprocessing_for_SLIC(data: np.ndarray):
    # 2️⃣ Clip extreme intensities (e.g. 0.5%–99.5% quantiles) for contrast enhancement
    vmin, vmax = np.quantile(data, (0.005, 0.995))
    data = np.clip(data, vmin, vmax)
    data = exposure.rescale_intensity(data, in_range=(vmin, vmax), out_range=(0, 1))  #  [oai_citation:0‡scikit-image.org](https://scikit-image.org/docs/0.25.x/api/skimage.segmentation.html?utm_source=chatgpt.com) [oai_citation:1‡researchgate.net](https://www.researchgate.net/publication/330691413_A_novel_technique_for_analysing_histogram_equalized_medical_images_using_superpixels?utm_source=chatgpt.com) [oai_citation:2‡arxiv.org](https://arxiv.org/abs/2204.05278?utm_source=chatgpt.com) [oai_citation:3‡scikit-image.org](https://scikit-image.org/skimage-tutorials/lectures/three_dimensional_image_processing.html?utm_source=chatgpt.com)

    sigma_vox = np.array([1.0 / s for s in spacing])  # blur by 1 mm across axes
    data = gaussian_filter(data, sigma=sigma_vox)

    data = exposure.equalize_hist(data)

    # Apply slice-wise CLAHE for 3D volume
    data = np.stack([equalize_adapthist(slice_, clip_limit=0.03)
                       for slice_ in data], axis=0)

    return data

In [1523]:
from skimage.segmentation import slic

In [1524]:
# Parametri SLIC: n_segments definisce quanti supervoxels circa si voglion
apply_SLIC = lambda data, spacing, n_supervoxels: slic(
                preprocessing_for_SLIC(data), 
                n_segments=n_supervoxels, 
                compactness=0.2,
                spacing=spacing,
                start_label=0,
                max_num_iter=10, 
                channel_axis=None)

### Initialize predictor

In [1525]:
# 2) Initialise predictor ------------------------
predictor = CustomNNUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=False, # == test time augmentation
    perform_everything_on_device=True,
    device=torch.device('cuda', 0),
    verbose=False,
    verbose_preprocessing=False,
    allow_tqdm=False #it interfere with SHAP loading bar
)
# initializes the network architecture, loads the checkpoint
predictor.initialize_from_trained_model_folder(
    model_dir,
    use_folds=(0,),
    checkpoint_name='checkpoint_final.pth',
)

/usr/local/lib/python3.10/site-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "


### some utils for Nifti geometry

In [1526]:
def get_spacing(ct_img_data: nib.nifti1.Nifti1Image):
    """affine = ct_img_data.affine
    spacing = np.sqrt(np.sum(affine[:3, :3] ** 2, axis=0))"""
    # this is equivalent to
    spacing = ct_img_data.header.get_zooms() # get_zooms returns (x, y, z) spacing
    return spacing

def get_origin(ct_img_data: nib.nifti1.Nifti1Image):
    affine = ct_img_data.affine
    origin = affine[:3, 3]
    return origin

# Define a ROI to explain segmentation in. 
Maybe this will provide a more useful attribution map, highlighting nearby organs

In [1527]:
"""# if true, we will use a json file containing the bounding box for the Region of Interest (ROI), otherwise we will use a masked segmentation
ROI_TYPE = "BoundingBox"  # "BoundingBox" or "MaskedSegmentation"""

'# if true, we will use a json file containing the bounding box for the Region of Interest (ROI), otherwise we will use a masked segmentation\nROI_TYPE = "BoundingBox"  # "BoundingBox" or "MaskedSegmentation'

In [1528]:
#BB_ROI = json.load(open(BB_ROI_paths[volume_codes[1]], "r")) if ROI_TYPE == "BoundingBox" else None

### ROI mask is a binary mask highlighting the lymphnodes of interest. We need a bounding box to crop the volume accordingly

In [1529]:
import nibabel as nib
import numpy as np

def get_mask_bbox_slices(mask_nii_path):
    """
    Load a binary ROI mask NIfTI and compute the minimal 3D bounding
    box slices containing all positive voxels.

    Parameters
    ----------
    mask_nii_path : str or Path
        Path to the input binary ROI mask NIfTI (.nii or .nii.gz).

    Returns
    -------
    bbox_slices : tuple of slice
        A 3-tuple of Python slice objects (x_slice, y_slice, z_slice)
        defining the minimal bounding box.
    """
    # 1) Load mask
    nii = nib.load(str(mask_nii_path))
    data = nii.get_fdata()
    if data.ndim != 3:
        raise ValueError("Input NIfTI must be a 3D volume")
    
    # 2) Find indices of positive voxels
    pos_voxels = np.argwhere(data > 0)
    if pos_voxels.size == 0:
        raise ValueError("No positive voxels found in mask")
    
    # 3) Compute min/max per axis
    x_min, y_min, z_min = pos_voxels.min(axis=0)
    x_max, y_max, z_max = pos_voxels.max(axis=0)
    
    # 4) Build slice objects (end is exclusive, hence +1)
    bbox_slices = (
        slice(int(x_min), int(x_max) + 1),
        slice(int(y_min), int(y_max) + 1),
        slice(int(z_min), int(z_max) + 1),
    )
    
    return bbox_slices

def get_slices_from_BB_ROI(BB_ROI: dict) -> Tuple[slice, slice, slice]:
    """
    Extracts the bounding box slices from the BB_ROI dictionary.
    Parameters
    ----------
    BB_ROI : dict
        Dictionary containing the bounding box coordinates with keys:
        example:
        {'FileFormat': array('MITK ROI', dtype='<U8'), 'Version': array(2), 'Geometry': array({'Size': [512.0, 512.0, 221.0], 'Transform': [1.3671875, -0.0, -0.0, 0, -0.0, 1.3671875, -0.0, 0, 0.0, 0.0, 5.0, 0, -350.0, -278.6000061035156, -432.239990234375, 1]},
      dtype=object), 'ROIs': array([{'ID': 0, 'Max': [370.5, 291.5, 114.5], 'Min': [276.5, 193.49999999999994, 92.5], 'Properties': {'ColorProperty': {'color': [1.0, 0.0, 0.0]}, 'StringProperty': {'name': 'AUTOMI_00004_0000 Bounding Box'}}}],
      dtype=object)}
    """
    # take the ceil of both min and max (ROI in MITK apprently is shifted by 0.5 voxels)
    x_slice = slice(int(np.ceil(BB_ROI['ROIs'][0]['Min'][0])), int(np.ceil(BB_ROI['ROIs'][0]['Max'][0])))
    y_slice = slice(int(np.ceil(BB_ROI['ROIs'][0]['Min'][1])), int(np.ceil(BB_ROI['ROIs'][0]['Max'][1])))
    z_slice = slice(int(np.ceil(BB_ROI['ROIs'][0]['Min'][2])), int(np.ceil(BB_ROI['ROIs'][0]['Max'][2])))
    return x_slice, y_slice, z_slice
   

In [1530]:
import json
from pathlib import Path
from typing import Tuple, Optional
import nibabel as nib


def load_roi_slices(
    ROI_type: str,
    ROI_BB_path: Optional[str] = None,
    ROI_segmentation_mask_path: Optional[str] = None
) -> Tuple[slice, slice, slice]:
    """
    Load ROI definition and return bounding-box slices in x, y, z order.

    Parameters
    ----------
    ROI_type : {"BoundingBox", "MaskedSegmentation"}
        Type of ROI to load.
    ROI_BB_path : str, optional
        Path to JSON file defining the bounding box ROI.
    ROI_segmentation_mask_path : str, optional
        Path to NIfTI file defining the ROI segmentation mask.

    Returns
    -------
    x_slice, y_slice, z_slice : slice
        Slices along each axis for cropping operations.
    """
    if ROI_type == "BoundingBox":
        if ROI_BB_path is None:
            raise ValueError("ROI_BB_path must be provided when ROI_type is 'BoundingBox'.")
        BB_ROI = json.load(open(ROI_BB_path, "r"))
        x_slice, y_slice, z_slice = get_slices_from_BB_ROI(BB_ROI)

    elif ROI_type == "MaskedSegmentation":
        if ROI_segmentation_mask_path is None:
            raise ValueError("ROI_segmentation_mask_path must be provided when ROI_type is 'MaskedSegmentation'.")
        # Optionally inspect mask details if needed:
        mask_img = nib.load(ROI_segmentation_mask_path)
        print(f"Loaded mask shape: {mask_img.shape}, affine: {mask_img.affine}")
        x_slice, y_slice, z_slice = get_mask_bbox_slices(ROI_segmentation_mask_path)

    else:
        raise ValueError(f"Unsupported ROI_type: {ROI_type}")

    return x_slice, y_slice, z_slice


### the identified region is our ROI bounding box in case we use the masked segmentation as manually derived ROI
### otherwise we just use the ROI bounding box

In [1531]:
import numpy as np

def slices_to_binary_mask(volume_shape, bbox_slices, dtype=np.uint8):
    """
    Create a binary mask of given shape where voxels inside the provided
    3D bounding‐box slices are set to 1, and all others to 0.

    Parameters
    ----------
    volume_shape : tuple of int
        The full 3D volume dimensions, e.g. (X, Y, Z).
    bbox_slices : tuple of slice
        A 3‐tuple of slice objects (x_slice, y_slice, z_slice) defining
        the region to mask.
    dtype : data‐type, optional
        The desired data‐type of the output mask (default: np.uint8).

    Returns
    -------
    mask : np.ndarray
        A binary mask array of shape `volume_shape`, with ones in the
        region defined by `bbox_slices` and zeros elsewhere.
    """
    if len(volume_shape) != len(bbox_slices):
        raise ValueError(f"volume_shape has {len(volume_shape)} dimensions, "
                         f"but bbox_slices has {len(bbox_slices)} slices")

    # Initialize mask to zeros
    mask = np.zeros(volume_shape, dtype=dtype)
    # Set the bounding‐box region to 1
    mask[bbox_slices] = 1

    return mask

In [1532]:
def create_roi_mask(
    volume_path: str,
    bbox_slices: Tuple[slice, slice, slice],
    output_path: Optional[str] = "ROI_binary_mask.nii.gz"
) -> str:
    """
    Create a binary ROI mask based on bounding-box slices and save as NIfTI.

    Parameters
    ----------
    volume_path : str
        Path to the input volume NIfTI file.
    bbox_slices : tuple of slice
        Slices (x_slice, y_slice, z_slice) defining the ROI bounding box.
    output_path : str, optional
        Path where the binary mask NIfTI will be saved.

    Returns
    -------
    output_path : str
        Path to the saved binary mask file.
    """
    img = nib.load(volume_path)
    volume_shape = img.get_fdata().shape
    affine = img.affine

    # Generate binary mask array
    mask_array = slices_to_binary_mask(
        volume_shape=volume_shape,
        bbox_slices=bbox_slices
    )

    # Create and save NIfTI mask
    save_nifty_binary(mask_array, affine, output_path)
    print(f"Saved binary ROI mask to {output_path}")

    return output_path

### to **crop** correctly the volume around the *ROI*, we need to derive the **receptive field** of the sliding window inference, that depends on the *patch size*.

In [1533]:
"""patch_size = np.array(predictor.configuration_manager.patch_size)
print("Patch size: ", patch_size)

# Receptive field is twice the patch size-1
RF = 2*(patch_size-1)
print("Receptive field: ", RF)"""

'patch_size = np.array(predictor.configuration_manager.patch_size)\nprint("Patch size: ", patch_size)\n\n# Receptive field is twice the patch size-1\nRF = 2*(patch_size-1)\nprint("Receptive field: ", RF)'

### Consider the receptive field to compute the final slices for cropping
Remember that model metadata are related to transposed volume (nnunetv2 takes (D, H, W) shape)

In [1534]:
def compute_rf_slices(
    bbox_slices: Tuple[slice, slice, slice],
    patch_size: np.ndarray,
    volume_shape: Tuple[int, int, int]
) -> Tuple[slice, slice, slice]:
    """
    Compute bounding-box slices expanded by the model's receptive field.

    RF is defined as 2*(patch_size - 1), and applied symmetrically.
    """
    x_slice, y_slice, z_slice = bbox_slices
    W, H, D = volume_shape

    # receptive field along each axis (model input axes reversed)
    RF = 2 * (patch_size - 1)
    RF_z, RF_y, RF_x = RF  # expect patch_size as [D,H,W]

    x_start = max(x_slice.start - RF_x // 2, 0)
    x_stop = min(x_slice.stop + RF_x // 2, W)
    y_start = max(y_slice.start - RF_y // 2, 0)
    y_stop = min(y_slice.stop + RF_y // 2, H)
    z_start = max(z_slice.start - RF_z // 2, 0)
    z_stop = min(z_slice.stop + RF_z // 2, D)

    return (slice(x_start, x_stop), slice(y_start, y_stop), slice(z_start, z_stop))

In [1535]:
import nibabel as nib
import numpy as np

def crop_volume_and_affine(nii_path, bbox_slices, save_cropped_nii_path=None):
    """
    Crop a 3D NIfTI volume using the given bounding-box slices and
    recompute the affine so the cropped volume retains correct world coordinates.

    Parameters
    ----------
    nii_path : str or Path
        Path to the input NIfTI volume (.nii or .nii.gz).
    bbox_slices : tuple of slice
        A 3-tuple (x_slice, y_slice, z_slice) as returned by get_mask_bbox_slices().
    save_cropped_nii_path : str or Path, optional
        If provided, the cropped volume will be saved here as a new NIfTI.

    Returns
    -------
    cropped_data : np.ndarray
        The volume data cropped to the bounding box.
    new_affine : np.ndarray
        The updated 4×4 affine transform for the cropped volume.
    """
    # 1) Load the original image
    img = nib.load(str(nii_path))
    data = img.get_fdata()
    affine = img.affine

    # 2) Crop the data array
    cropped_data = data[bbox_slices]

    # 3) Extract the voxel‐offsets for x, y, z from the slice starts
    x_slice, y_slice, z_slice = bbox_slices
    z0, y0, x0 = z_slice.start, y_slice.start, x_slice.start

    # 4) Compute the new affine translation: shift the origin by the voxel offsets
    # Note voxel coordinates are (i, j, k) = (x, y, z)
    offset_vox = np.array([x0, y0, z0])
    new_affine = affine.copy()
    new_affine[:3, 3] += affine[:3, :3].dot(offset_vox)

    # 5) Optionally save the cropped volume
    if save_cropped_nii_path is not None:
        cropped_img = nib.Nifti1Image(cropped_data, new_affine)
        nib.save(cropped_img, str(save_cropped_nii_path))

    return cropped_data, new_affine


### We need a way to check original mask overlapping in the new cropped volume

### We also need a mask to correctly ignoring out-of ROI context in our aggregation metrics

In [1536]:
from typing import Tuple, Optional, Dict

def crop_volume_with_rf(
    volume_path: str,
    bbox_slices: Tuple[slice, slice, slice],
    patch_size: np.ndarray,
    output_dir: Optional[str] = "."
) -> Dict[str, Tuple[np.ndarray, np.ndarray]]:
    """
    Crop the volume and binary ROI mask to the ROI padded by receptive field.

    Returns a dict with keys:
      - 'cropped_volume'
      - 'affine_cropped_volume'
      - 'cropped_roi_mask'
      - 'affine_cropped_mask'
    """
    volume_img = nib.load(volume_path)
    volume_shape = volume_img.get_fdata().shape

    # compute RF-expanded slices
    padded_slices = compute_rf_slices(bbox_slices, patch_size, volume_shape)

    # ensure output directory exists
    out_dir = Path(output_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # crop volume
    cropped_vol_path = out_dir / "cropped_volume.nii.gz"
    cropped_volume, affine_cropped_volume = crop_volume_and_affine(
        nii_path=volume_path,
        bbox_slices=padded_slices,
        save_cropped_nii_path=cropped_vol_path
    )

    # crop ROI mask
    roi_mask_path = out_dir / "ROI_binary_mask.nii.gz"
    cropped_mask, affine_cropped_mask = crop_volume_and_affine(
        nii_path=str(roi_mask_path),
        bbox_slices=padded_slices,
        save_cropped_nii_path=out_dir / "cropped_mask_with_RF.nii.gz"
    )

    return {
        "cropped_volume": (cropped_volume, affine_cropped_volume),
        "cropped_roi_mask": (cropped_mask, affine_cropped_mask)
    }

## We execute SHAP on this cropped image, and we only consider our ROI

### set device

In [1537]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Preprocessing (skipped for now)

In [1538]:
#NNUNET_PREPROCESSING = False

In [1539]:
def preprocess_volume(
    cropped_volume_path: str,
    predictor,
    dataset_json_path: str,
    use_nnunet: bool = True
) -> Tuple[np.ndarray, nib.Nifti1Image]:
    """
    Preprocess the cropped volume for nnU-Net inference.

    If `use_nnunet` is True, applies nnU-Net's default preprocessing.
    Otherwise, loads the volume directly and reformats axes.

    Returns
    -------
    volume_np : np.ndarray
        Array of shape (1, C, D, H, W) ready for torch conversion.
    saved_volume_nii : nib.Nifti1Image
        NIfTI image object of the preprocessed volume (for debugging or saving).
    """
    if use_nnunet:
        # nnU-Net preprocessing utility, assumed imported
        volume_np = nnunetv2_default_preprocessing(
            cropped_volume_path,
            predictor,
            dataset_json_path
        )  # expects (C, D, H, W)
        # Save NIfTI for debugging
        volume_transposed = volume_np.squeeze().transpose(2, 1, 0) # back to (W,H,D)
        affine = nib.load(cropped_volume_path).affine
        path = Path(cropped_volume_path).replace(suffix="_preproc.nii.gz")
        save_nifty(volume_transposed, affine, path)

    else:
        cropped = nib.load(cropped_volume_path)
        data = cropped.get_fdata()
        data = np.transpose(data, (2, 1, 0))  # (D,H,W)
        data = np.expand_dims(data, axis=0)   # (1, D,H,W)
        volume_np = data
        volume_nii = None

    return volume_np, volume_nii

### Supervoxels subdivision

In [1540]:
def generate_supervoxel_map(
    cropped_volume_path: str,
    supervoxel_type: str = "FCC",
    fcc_cube_side: float = 100.0,
    slic_n_supervoxels: int = 380,
    organ_map_path: Optional[str] = None,
    just_load: Optional[bool] = False
) -> np.ndarray:
    """
    Generate a supervoxel map for the cropped volume.
    Supports:
      - "full-organs": 1 organ = 1 supervoxel, simple as that
      - "FCC": standard face-centered cubic
      - "SLIC": simple linear iterative clustering
      - "FCC-organs": FCC constrained by organ presence
    Returns an array of shape (D, H, W) with consecutive integer labels.
    """
    if just_load:
        sv_map = nib.load("supervoxel_map.nii.gz").get_fdata()
    else:
        img = nib.load(cropped_volume_path)
        data = img.get_fdata().astype(np.float32)

        if supervoxel_type == "full-organs":
            if organ_map_path is None:
                raise ValueError("organ_map_path must be provided for 'full-organs' type")
            # load organ segmentation map
            organ_img = nib.load(organ_map_path)
            # the supervoxel map is just the organ map
            sv_map = organ_img.get_fdata().astype(np.float32)
        elif supervoxel_type == "FCC":
            sv_map = generate_FCC_supervoxel_map(img, S=fcc_cube_side)
        elif supervoxel_type == "SLIC":
            spacing = get_spacing(data)
            sv_map = apply_SLIC(data, spacing, slic_n_supervoxels)
        elif supervoxel_type == "FCC-organs":
            if organ_map_path is None:
                raise ValueError("organ_map_path must be provided for 'FCC-organs' type")
            # load organ segmentation map
            organ_img = nib.load(organ_map_path)
            sv_map, organ_table, statistics = generate_FCC_organs_supervoxel_map(
            volume_img=img, organ_img=organ_img, S=fcc_cube_side
        )
        else:
            raise ValueError(f"Unsupported supervoxel_type: {supervoxel_type}")

        # remap labels to consecutive ints
        _, inverse = np.unique(sv_map, return_inverse=True)
        sv_map = inverse.reshape(sv_map.shape)
        
        # debug
        image = nib.Nifti1Image(sv_map.astype(np.float32), img.affine)
        nib.save(image, "supervoxel_map.nii.gz")
        print("supervoxel map saved!")

    # reorder axes (W,H,D) -> (D,H,W)
    sv_map = np.transpose(sv_map, (2, 1, 0))
    return sv_map

### derive baseline cached dictionary

In [1541]:
def get_cached_output_dictionary(volume_file: Path,
                                 predictor: CustomNNUNetPredictor,
                                 preprocess_before_run: bool = True,
                                verbose: bool = False) -> dict:
        """
        Return a dictionary indexed by the slices for the sliding window, of the output of the inference for each patch
        of the original volume
        """
        rw = predictor.plans_manager.image_reader_writer_class()

        # If nnU-Net returns a class instead of an instance, instantiate it
        if callable(rw) and not hasattr(rw, "read_images"):
            rw = rw()

        orig_image, orig_props = rw.read_images(
            [str(volume_file)]
        )             # (C, Z, Y, X)

        if preprocess_before_run:
        
            preprocessor = predictor.configuration_manager.preprocessor_class()
            # the following cause the kernel death at first notebook run
            data_pp, _, _ = preprocessor.run_case_npy(
                    orig_image,
                    seg=None,
                    properties=orig_props,
                    plans_manager=predictor.plans_manager,
                    configuration_manager=predictor.configuration_manager,
                    dataset_json=predictor.dataset_json
                )
        
            # to torch, channel-first is already true
            inp_tensor = torch.from_numpy(data_pp)
        else:
            inp_tensor = torch.from_numpy(orig_image)

        slicers = predictor._internal_get_sliding_window_slicers(inp_tensor.shape[1:])

        dictionary = predictor.get_output_dictionary_sliding_window(inp_tensor, slicers)

        return dictionary

In [1542]:
#USE_STORED_DICTIONARY = False

In [1543]:
def compute_baseline_prediction(
    volume_tensor: "torch.Tensor",
    predictor,
    cropped_volume_path: str,
    use_nnunet: bool = True
) -> Tuple["torch.Tensor", Dict]:
    """
    Compute baseline segmentation mask and cache full prediction outputs.

    Returns the segmentation mask tensor and a cache dictionary.
    """
    # Predict logits
    logits = predictor.predict_sliding_window_return_logits(volume_tensor[0])
    # Convert to binary mask (assuming class 1 is positive)
    seg_mask = (torch.argmax(logits, dim=0) == 1)

    # Cache prediction outputs for SHAP
    cache_dict = get_cached_output_dictionary(
        volume_file=cropped_volume_path,
        predictor=predictor,
        preprocess_before_run=use_nnunet,
        verbose=True
    )
    return seg_mask, cache_dict

### Include masking in the forward function

### **Chrabaszcz aggregation**  

Let

* $z_1^{(i)}(x)$ – class-1 logit at voxel $x$ after perturbation *i*
* $P_i(x)=\mathbf 1\!\left[\arg\max_c z_c^{(i)}(x)=1\right]$ – binary mask of voxels currently predicted as lymph-node
* no ROI, total volume considered

$$
S_{\text{Chr}}^{(i)} \;=\;
\frac{1}{\alpha}\sum_{x} P_i(x)\;z_1^{(i)}(x)
$$

where $\alpha$ is the `scaling_factor`.

* **Counts evidence only from voxels the model *currently* labels as class 1.**
* *False positives (FP):* contribute **positively** (they are in $P_i$).
* *False negatives (FN):* contribute **zero** (their logit is absent).

Source: Chrabaszcz et al., *Aggregated Attributions for Explanatory Analysis of 3-D Segmentation Models*, 2024.


In [1544]:
def chrabaszcz_aggregation(logits: torch.Tensor,
                           scaling_factor: float = 1.0,
                          ) -> torch.Tensor:
    """
    aggregate the output logits in a sum, following the proposed method in "Chrabaszcz et al. - 2024 - Aggregated Attributions for
    Explanatory Analysis of 3D Segmentation Models"
    """
    seg_mask = (torch.argmax(logits, dim=0) == 1)
    aggregate = torch.sum(logits[1].double() * seg_mask)

    return aggregate / scaling_factor  # normalize to avoid overflows in SHAP


### **True positive aggregation** (Chrabaszcz aggregation + baseline-mask filtering)

Introduce the unperturbed prediction $P_0$, and, optionally, the ROI mask $R(x)$.
Keep only voxels that are **still** class 1 *and* were class 1 before:

$$
S_{\text{Chr\,keep}}^{(i)} \;=\;
\frac{1}{\alpha}\sum_{x} \bigl[P_i(x)\land P_0(x)\land R(x)\bigr]\;z_1^{(i)}(x)
$$

* **True positives preserved** (TP core) add positive evidence.
* **FP created by the perturbation** are **ignored** (masked out).
* **FN** lower the score indirectly because their logits disappear from the sum.

Conceptually this is the **positive part** of a signed logit-difference metric.

In [1545]:
def true_positive_aggregation(logits: torch.Tensor,
                          unperturbed_binary_mask: torch.Tensor,
                          ROI_mask: torch.Tensor,
                           scaling_factor: float = 1.0,
                          ) -> torch.Tensor:
    """
    aggregate the output logits in a sum, following the proposed method in "Chrabaszcz et al. - 2024 - Aggregated Attributions for
    Explanatory Analysis of 3D Segmentation Models", with the  addition of filtering by the unperturbed segmentation.
    We can use this to ignore "false positive" voxels -> only account for true positive contribution;
    so this corresponds conceptually to the positive part of a logits difference metric
    """
    seg_mask = (torch.argmax(logits, dim=0) == 1)          # (D,H,W)
    seg_mask = seg_mask.bool() & ROI_mask.bool() & unperturbed_binary_mask.bool()  # prefer boolean indexing for reletively sparse tensors
    aggregate = torch.sum(logits[1].double()[seg_mask])

    return aggregate / scaling_factor  # normalize to avoid overflows in SHAP

---

### **False-positive aggregation**

Directly sum class-1 evidence from **new** positives inside ROI:

$$
S_{\text{FP}}^{(i)} \;=\;
\frac{1}{\alpha}\sum_{x}
\bigl[P_i(x)\land\neg P_0(x)\land R(x)\bigr]\;z_1^{(i)}(x)
$$

* Measures **only** the spurious lymph-node evidence a perturbation introduces.
* Higher value ⇒ stronger tendency to hallucinate extra nodes.

In [1546]:
def false_positive_aggregation(logits: torch.Tensor,
                              unperturbed_binary_mask: torch.Tensor,
                              ROI_mask: torch.Tensor,
                              scaling_factor: float = 1.0,
                              ) -> torch.Tensor:
    """
    Negative part of signed logit-difference objective, returned with positive sign;
    Only accounts for false positive voxels in segmentation (spurious lymph nodes)
    """
    # current segmentation (prevailing class)
    seg_mask = (torch.argmax(logits, dim=0) == 1)     # (D,H,W) ∈ {0,1}

    fp_mask  = seg_mask * ROI_mask * torch.logical_not(unperturbed_binary_mask.bool()).float()   # prefer float multiplication for dense tensors
    aggregate = torch.sum(logits[1].double() * fp_mask) 

    return aggregate / scaling_factor

---

### **Dice aggregation (prediction-vs-baseline, ROI-restricted)**

Let $R(x)$ be the ROI mask.

$$
P_i' = P_i \odot R, \qquad
P_0' = P_0 \odot R
$$

$$
S_{\text{Dice}}^{(i)} \;=\;
\frac{1}{\alpha}\;
\frac{2\,\langle P_i',\,P_0'\rangle}{\lVert P_i'\rVert_1 + \lVert P_0'\rVert_1 + \varepsilon}
$$

* Drops when either **FP** ($P_i'=1,\,P_0'=0$) or **FN** ($P_i'=0,\,P_0'=1$) appear → penalises both error types symmetrically.

Based on the “self-consistency Dice” used in MiSuRe (Hasany et al., 2024).





In [1547]:
def dice_aggregation(logits: torch.Tensor,
                    unperturbed_binary_mask: torch.Tensor,
                    ROI_mask: torch.Tensor,
                    scaling_factor: float = 1.0,
                    eps: float = 1e-9,  # small value to avoid division by zero
                    ) -> torch.Tensor:
    """
    Use Dice score, the same aggregation measure from "Hasany et al. - 2024 - MiSuRe is all you need to explain your image segmentation"
    Dice score provides a single aggregation metric that accounts for both false negatives and false positives penalization.
    Specificly, we instead score each perturbation supervoxels by that Dice => supervoxels that contribute the most in reproducing
    the baseline segmentation, will get an higher score
    """
    # 1. Boolean masks restricted to ROI
    pred = (logits.argmax(dim=0) == 1).float() * ROI_mask.float()
    base = unperturbed_binary_mask.float()      * ROI_mask.float()

    # 2. Intersection and denominator
    inter = (pred * base).sum()
    denom = pred.sum() + base.sum() + eps       # |P| + |B|

    # 3. Dice coefficient
    dice = (2.0 * inter) / denom

    return dice / scaling_factor

---

### **Signed logit-difference (masked)**

Define a signed weight

$$
w(x)=
\begin{cases}
+1 & \text{if } P_0(x)=1\\
-1 & \text{otherwise}
\end{cases},
\qquad w(x)\leftarrow w(x)\,R(x)
$$

$$
S_{\text{LD}}^{(i)} \;=\;
\frac{1}{\alpha}\sum_{x} P_i(x)\;w(x)\;z_1^{(i)}(x)
$$

* **Positive attribution:** voxels that *keep* the baseline TP (support segmentation).
* **Negative attribution:** voxels that become class 1 **only** after perturbation (generate FP inside ROI).
* FN reduce the positive term (logits disappear) but do **not** add negative mass.

In [1548]:


def logit_difference_aggregation(
        logits: torch.Tensor,
        unperturbed_binary_mask: torch.Tensor,
        ROI_mask: torch.Tensor,
        scaling_factor: float = 1.0,
) -> torch.Tensor:
    """
    Signed logit-difference objective *masked by the prevailing class*.
    """
    # current segmentation (prevailing class)
    seg_mask = (torch.argmax(logits, dim=0) == 1)     # (D,H,W) ∈ {0,1}

    # +1 inside baseline positives, −1 elsewhere...
    signed_weight = torch.where(unperturbed_binary_mask.bool(),
                                torch.tensor(1.0, device=logits.device),
                                torch.tensor(-1.0, device=logits.device))

    # ... but we only care of false positives inside the ROI (we don't even have the segmentation mask outside the ROI)
    signed_weight = signed_weight * ROI_mask

    # aggregate signed class-1 evidence, restricted to voxels
    # that are *currently* predicted as class-1 (seg_mask)
    aggregate = torch.sum(logits[1] * seg_mask * signed_weight)
    return aggregate / scaling_factor

### Define a function to prepare all the steps for SHAP

In [1549]:
from pathlib import Path
from typing import Tuple, Optional, Dict

import numpy as np
import nibabel as nib
import torch

def prepare_data_for_shap(
    volume_path: str,
    patch_size: np.ndarray,
    predictor,
    dataset_json_path: str,
    ROI_BB_path: Optional[str] = None,
    ROI_segmentation_mask_path: Optional[str] = None,
    ROI_type: str = "BoundingBox",
    nnunet_preprocessing: bool = True,
    supervoxel_type: str = "FCC",
    fcc_supervoxel_size: Optional[float] = None,
    slic_n_supervoxels: Optional[int] = None,
    organ_map_path: Optional[str] = None,
    load_stored_sv_map: Optional[bool] = False, # Forbidden for multiple (different) volumes, only for debugging single volume
    device: str = 'cuda:0'
) -> Tuple[torch.Tensor, np.ndarray, torch.Tensor, torch.Tensor, torch.Tensor, Dict]:
    """
    Orchestrate data preparation by invoking modular utilities.
    Returns:
      - volume_tensor (1,C,D,H,W)
      - affine of cropped volume
      - supervoxel_map_tensor (D,H,W)
      - segmentation_mask_tensor (D,H,W)
      - ROI_mask_tensor (D,H,W)
      - cache_dict
    """
    # 1. Load ROI slices
    x_slice, y_slice, z_slice = load_roi_slices(
        ROI_type,
        ROI_BB_path,
        ROI_segmentation_mask_path
    )

    # 2. Create binary ROI mask
    roi_mask_path = create_roi_mask(
        volume_path,
        (x_slice, y_slice, z_slice)
    )

    # 3. Compute receptive-field–padded slices
    vol_img = nib.load(volume_path)
    padded_slices = compute_rf_slices(
        (x_slice, y_slice, z_slice),
        patch_size,
        vol_img.get_fdata().shape
    )

    # 4. Crop volume and ROI mask using RF slices
    cropped = crop_volume_with_rf(
        volume_path,
        (x_slice, y_slice, z_slice),
        patch_size
    )
    cropped_volume, affine_cropped = cropped['cropped_volume']
    cropped_mask, _ = cropped['cropped_roi_mask']

    cropped_vol_path = Path('cropped_volume.nii.gz')
    cropped_mask_path = Path('cropped_mask_with_RF.nii.gz')

    # 5. If provided, also crop the organs map with the same RF slices
    cropped_organs_path = None
    if organ_map_path is not None:
        cropped_organs_path = 'cropped_organ_map_with_RF.nii.gz'
        crop_volume_and_affine(
            organ_map_path,
            padded_slices,
            Path(cropped_organs_path)
        )

    # 6. Preprocess volume
    volume_np, _ = preprocess_volume(
        str(cropped_vol_path),
        predictor,
        dataset_json_path,
        use_nnunet=nnunet_preprocessing
    )

    # 7. Convert to torch tensor
    volume_tensor = torch.from_numpy(
        volume_np.astype(np.float32)
    ).unsqueeze(0).to(device)

    # 8. Generate supervoxel map, passing cropped_organs_path for FCC-organs
    sv_array = generate_supervoxel_map(
        str(cropped_vol_path),
        supervoxel_type=supervoxel_type,
        fcc_cube_side = fcc_supervoxel_size,
        slic_n_supervoxels = slic_n_supervoxels,
        organ_map_path=cropped_organs_path,
        just_load=load_stored_sv_map
    )
    sv_tensor = torch.from_numpy(sv_array).long().to(device)

    # 9. Compute baseline segmentation and cache
    seg_mask, cache_dict = compute_baseline_prediction(
        volume_tensor,
        predictor,
        str(cropped_vol_path),
        use_nnunet=nnunet_preprocessing
    )

    # 10. Prepare ROI mask tensor
    roi_mask_tensor = torch.from_numpy(
        np.transpose(cropped_mask, (2, 1, 0))
    ).to(device)

    return (
        volume_tensor,
        affine_cropped,
        sv_tensor,
        seg_mask.to(device),
        roi_mask_tensor,
        cache_dict
    )

## Iterate over volumes and run SHAP

In [1550]:
# some parameters for the experiment
N_SAMPLES = 2000

# if true, we will use a json file containing the bounding box for the Region of Interest (ROI), otherwise we will use a masked segmentation
ROI_TYPE = "BoundingBox"  # "BoundingBox" or "MaskedSegmentation"

NNUNET_PREPROCESSING = False
SUPERVOXEL_TYPE = "FCC-organs"
FCC_SUPERVOXEL_SIZE = 50 #[mm]
SLIC_N_SUPERVOXELS = 380

WANDB = True

In [1551]:
import os
import wandb

# evita spam in console
os.environ.setdefault("WANDB_SILENT", "true")

def wandb_login(project=None, entity=None):
    # 1) prova da variabile d’ambiente / .env
    api_key = os.getenv("WANDB_API_KEY")

    # 2) fallback a kaggle_secrets se non trovato
    if not api_key:
        try:
            from kaggle_secrets import UserSecretsClient
            api_key = UserSecretsClient().get_secret("wandb_key")
        except Exception:
            raise RuntimeError("WANDB_API_KEY non trovata né in .env né in Kaggle secrets")

    # login
    wandb.login(key=api_key, relogin=True)

    # opzionale: avvia run
    if project is not None and entity is not None: 
        return wandb.init(project=project, entity=entity, settings=wandb.Settings(silent=True))


In [1552]:
if WANDB:
    os.environ["WANDB_SILENT"] = "true"
    settings=wandb.Settings(silent=True)  # no console spam
    
    wandb_login()

In [1553]:
dataset_json_path = Path(model_dir) / "dataset.json"

In [1554]:
if ROI_TYPE == "BoundingBox":
    BB_ROI_paths = {}
    for volume_code in volume_codes:
        if IN_KAGGLE:
            BB_ROI_paths[volume_code] = f"{mount_dir}/BB-ROI/AUTOMI_{volume_code}.json"
        else:
            BB_ROI_paths[volume_code] = nnUNet_results + "/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/BB-ROI/" + f"AUTOMI_{volume_code}.json"
elif ROI_TYPE == "MaskedSegmentation":
    # get the manually derived ROI mask from the dataset, where we manually added it
    if IN_KAGGLE:
        ROI_segmentation_mask_path = "/kaggle/input/segmentation-masked-ROI.nii"
    else:
        ROI_segmentation_mask_path = nnUNet_raw + "/segmentation-masked-ROI.nii"

In [1555]:
patch_size = np.array(predictor.configuration_manager.patch_size)

In [1556]:
@torch.inference_mode()
def forward_segmentation_output_to_explain(
        input_image:         torch.Tensor,
        perturbation_mask:   torch.BoolTensor | None,
        segmentation_mask:      torch.Tensor,   # remember that must be cropped to the same size of the other tensors
        ROI_bounding_box_mask:      torch.Tensor,
        baseline_prediction_dict: dict,
        aggregation_fn: Callable = true_positive_aggregation,
) -> torch.Tensor:           # returns a scalar per sample
    """
    Example aggregate: sum of lymph-node logits (class 1) in the mask produced
    by the network – adapt to your real metric as needed.
    """
    logits = predictor.predict_sliding_window_return_logits_with_caching(
        input_image, perturbation_mask, baseline_prediction_dict,
    )                              # (C, D, H, W)
    # we now mask both by the segmentation prevalent class, and by ROI
    D,H,W = logits.shape[1:]
    aggregate = aggregation_fn(
        logits = logits,
        unperturbed_binary_mask = segmentation_mask,
        ROI_mask = ROI_bounding_box_mask,
        scaling_factor = ((D*H*W) if aggregation_fn != dice_aggregation else 1.0)
    )

    return aggregate

In [1557]:
volume_codes = ["00004", "00005", "00024", "00027", "00029", "00034", "00039", "00044"]
#volume_codes = ["00004"]
#volume_codes = ["00039"]

In [1558]:
#aggregation_functions = [true_positive_aggregation, false_positive_aggregation, dice_aggregation, logit_difference_aggregation]
aggregation_functions = [dice_aggregation, logit_difference_aggregation]

In [ ]:
import pickle

for i, volume_code in enumerate(volume_codes):
    print(f"Working on volume code {i}: {volume_code}")
    print(f"  Shape: {nib.load(ct_img_paths[volume_code]).shape}")

    os.mkdir(volume_code) if not os.path.exists(volume_code) else None

    volume, affine, supervoxel_map, segmentation_mask, ROI_mask, cache_dict = prepare_data_for_shap(
        volume_path=ct_img_paths[volume_code],
        patch_size=patch_size,
        predictor=predictor,
        dataset_json_path=dataset_json_path,
        ROI_BB_path=BB_ROI_paths[volume_code] if ROI_TYPE == "BoundingBox" else None,
        ROI_segmentation_mask_path=ROI_segmentation_mask_path if ROI_TYPE == "MaskedSegmentation" else None,
        ROI_type=ROI_TYPE,
        nnunet_preprocessing=NNUNET_PREPROCESSING,
        supervoxel_type=SUPERVOXEL_TYPE,
        fcc_supervoxel_size=FCC_SUPERVOXEL_SIZE,
        slic_n_supervoxels=SLIC_N_SUPERVOXELS,
        organ_map_path=organ_map_paths[volume_code],
        device=device,
    )

    
    for agg_fn in aggregation_functions:
        print(f"  Using aggregation function: {agg_fn.__name__}")

        if WANDB:
            # Detect device name
            if device.type == "cuda":
                GPU_NAME = torch.cuda.get_device_name(device)
            else:
                GPU_NAME = "CPU"
                
            wandb_run = wandb.init(
                project="automi",  # your W&B project name
                #name="run_01",                 # optional, name for this run
                config={
                        "device_name": GPU_NAME,
                        "volume_code": volume_code,
                        "n_samples": N_SAMPLES,
                        "supervoxel_type": SUPERVOXEL_TYPE,
                        "fcc_supervoxel_size": FCC_SUPERVOXEL_SIZE,
                        "slic_n_supervoxels": SLIC_N_SUPERVOXELS,
                        "nnunet_preprocessing": NNUNET_PREPROCESSING,
                        "aggregation_function": agg_fn.__name__
                    }
            )


        # a) wrap the aggregation function
        forward_func = lambda vol, mask: forward_segmentation_output_to_explain(
            input_image=vol,
            perturbation_mask=mask,
            segmentation_mask=segmentation_mask,
            ROI_bounding_box_mask=ROI_mask,
            baseline_prediction_dict=cache_dict,
            aggregation_fn=agg_fn
        )

        # b) create the explainer
        explainer = KernelShapWithMask(forward_func=forward_func)

        # c) wrap your cached‐forward method:
        explainer = KernelShapWithMask(
            forward_func=lambda vol, _perturbation_mask: forward_segmentation_output_to_explain(
                input_image=vol,
                perturbation_mask=_perturbation_mask,
                segmentation_mask=segmentation_mask,
                ROI_bounding_box_mask=ROI_mask,
                baseline_prediction_dict=cache_dict)
        )

        # d) compute SHAP
        attr = explainer.attribute(
            inputs=volume,       # (1,C,D,H,W)
            baselines=0.0, 
            feature_mask=supervoxel_map,
            attribute_background=(SUPERVOXEL_TYPE not in ["FCC-organs", "full-organs"]),
            n_samples=N_SAMPLES,
            return_input_shape=True,
            monitor_log_path=None,#join(volume_code, "monitor.jsonl"),
            monitor_convergence_step=None,#10,
            monitor_local_accuracy_step=None,#20,
            show_progress=True,
        )
        print("Attributions:", attr.shape)  # → (1,C,D,H,W)

        os.makedirs(join(SUPERVOXEL_TYPE, agg_fn.__name__, volume_code), exist_ok=True)
        with open(join(SUPERVOXEL_TYPE, agg_fn.__name__, volume_code, 'dataset.pkl'), 'wb') as file:
            pickle.dump(explainer.dataset, file)

        attr_postprocessed = attr[0][0].detach().cpu().numpy().transpose(2,1,0) # (W, H, D)
        attr_img = nib.Nifti1Image(attr_postprocessed, affine)
        nib.save(attr_img, join(SUPERVOXEL_TYPE, agg_fn.__name__, volume_code, 'attribution_map.nii.gz'))

        if WANDB:
            wandb_run.finish()



Working on volume code 0: 00004
  Shape: (512, 512, 221)
Saved binary ROI mask to ROI_binary_mask.nii.gz
Generated 2760 FCC centers
Assigning voxels to FCC centers...
Creating organ-aware supervoxels...
Created 427 organ-aware supervoxels
supervoxel map saved!
  Using aggregation function: dice_aggregation
